In [30]:
import gym
# Tensorflow Library
import tensorflow as tf
import numpy as np
from collections import deque
from time import time, strftime, localtime
import os
tf.get_logger().setLevel('ERROR')
import matplotlib.pyplot as plt

In [31]:
def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

def initiate_NN2(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

def evaluate(model,test_episodes=100):
    test_rewards=[]
    for i in range(test_episodes):
        rewards=0
        done = False
        state=env.reset()
        while not done:
            Y=model.predict(state.reshape(1,-1))
            action = Y[0].tolist().index(np.max(Y))
            new_state, reward, done, info = env.step(action)
            rewards+=reward
            state=new_state
        test_rewards.append(rewards)
        #print(i,rewards)
    return test_rewards

env = gym.make('LunarLander-v2')
# DQN COPY2
max_e = 1
min_e=0.0001
decay_rate=0.002
learning_rate=0.0002
replay_batch_size=30
gamma = 0.99
model = initiate_NN(learning_rate)
model_string='_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(decay_rate)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)
model.load_weights('weights'+model_string+'.h5')

test_rewards =evaluate(model)
import pickle

with open('rewards/test'+model_string+'.json', 'wb') as fp:
    pickle.dump({'test_rewards':test_rewards}, fp)

np.average(test_rewards)

KeyboardInterrupt: 

In [2]:
!mkdir test

A subdirectory or file test already exists.


In [2]:
import gym, random, tempfile
import numpy as np
from gym import wrappers
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.max_e = 1
        self.min_e = 0.0001
        self.epsilon_decay = 0.002
        self.gamma = 0.99
        self.learning_rate = 0.0002
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()


    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    tdir = tempfile.mkdtemp()
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.min_e:
            agent.epsilon *= agent.epsilon_decay
            agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()




episode:  0  score:  -129.00  avg_score:  -129.00  frames:  96  epsilon:  1.00
episode:  1  score:  -166.53  avg_score:  -147.77  frames:  66  epsilon:  1.00
episode:  2  score:  -139.10  avg_score:  -144.88  frames:  86  epsilon:  1.00
episode:  3  score:  -67.02  avg_score:  -125.41  frames:  71  epsilon:  0.99
episode:  4  score:  -46.29  avg_score:  -109.59  frames:  59  epsilon:  0.99
episode:  5  score:  -120.50  avg_score:  -111.41  frames:  99  epsilon:  0.99
episode:  6  score:  -100.33  avg_score:  -109.83  frames:  78  epsilon:  0.99
episode:  7  score:  -128.64  avg_score:  -112.18  frames:  80  epsilon:  0.99
episode:  8  score:  -20.72  avg_score:  -102.02  frames:  92  epsilon:  0.98
episode:  9  score:  -86.17  avg_score:  -100.43  frames:  66  epsilon:  0.98
episode:  10  score:  -415.47  avg_score:  -129.07  frames:  108  epsilon:  0.98
episode:  11  score:  -192.95  avg_score:  -134.39  frames:  89  epsilon:  0.98
episode:  12  score:  -103.70  avg_score:  -132.03  f

episode:  103  score:  -82.48  avg_score:  -167.16  frames:  71  epsilon:  0.81
episode:  104  score:  -146.75  avg_score:  -168.17  frames:  87  epsilon:  0.81
episode:  105  score:  -117.10  avg_score:  -168.13  frames:  144  epsilon:  0.81
episode:  106  score:  -106.12  avg_score:  -168.19  frames:  70  epsilon:  0.81
episode:  107  score:  -52.67  avg_score:  -167.43  frames:  147  epsilon:  0.81
episode:  108  score:  -66.55  avg_score:  -167.89  frames:  82  epsilon:  0.81
episode:  109  score:  -44.69  avg_score:  -167.47  frames:  87  epsilon:  0.80
episode:  110  score:  -145.66  avg_score:  -164.77  frames:  131  epsilon:  0.80
episode:  111  score:  -102.73  avg_score:  -163.87  frames:  96  epsilon:  0.80
episode:  112  score:  -148.38  avg_score:  -164.32  frames:  119  epsilon:  0.80
episode:  113  score:  -122.89  avg_score:  -162.78  frames:  102  epsilon:  0.80
episode:  114  score:  -129.17  avg_score:  -161.31  frames:  142  epsilon:  0.80
episode:  115  score:  -12

episode:  205  score:  -100.88  avg_score:  -83.04  frames:  122  epsilon:  0.66
episode:  206  score:  -81.06  avg_score:  -82.79  frames:  138  epsilon:  0.66
episode:  207  score:  -107.16  avg_score:  -83.34  frames:  64  epsilon:  0.66
episode:  208  score:  -1.05  avg_score:  -82.68  frames:  181  epsilon:  0.66
episode:  209  score:  -88.18  avg_score:  -83.12  frames:  119  epsilon:  0.66
episode:  210  score:  -11.10  avg_score:  -81.77  frames:  183  epsilon:  0.66
episode:  211  score:  -72.89  avg_score:  -81.47  frames:  105  epsilon:  0.66
episode:  212  score:  -100.41  avg_score:  -80.99  frames:  90  epsilon:  0.65
episode:  213  score:  -94.84  avg_score:  -80.71  frames:  104  epsilon:  0.65
episode:  214  score:  -46.90  avg_score:  -79.89  frames:  125  epsilon:  0.65
episode:  215  score:  -66.24  avg_score:  -79.26  frames:  150  epsilon:  0.65
episode:  216  score:  -7.54  avg_score:  -78.02  frames:  231  epsilon:  0.65
episode:  217  score:  -183.96  avg_score

episode:  308  score:  -49.64  avg_score:  -50.56  frames:  167  epsilon:  0.54
episode:  309  score:  -6.24  avg_score:  -49.74  frames:  202  epsilon:  0.54
episode:  310  score:  -28.37  avg_score:  -49.91  frames:  116  epsilon:  0.54
episode:  311  score:  21.72  avg_score:  -48.97  frames:  348  epsilon:  0.54
episode:  312  score:  -23.99  avg_score:  -48.20  frames:  158  epsilon:  0.54
episode:  313  score:  -31.63  avg_score:  -47.57  frames:  128  epsilon:  0.53
episode:  314  score:  -16.88  avg_score:  -47.27  frames:  107  epsilon:  0.53
episode:  315  score:  -290.47  avg_score:  -49.51  frames:  589  epsilon:  0.53
episode:  316  score:  -18.99  avg_score:  -49.63  frames:  999  epsilon:  0.53
episode:  317  score:  -34.47  avg_score:  -48.13  frames:  129  epsilon:  0.53
episode:  318  score:  -10.88  avg_score:  -48.15  frames:  119  epsilon:  0.53
episode:  319  score:  26.59  avg_score:  -47.25  frames:  168  epsilon:  0.53
episode:  320  score:  -37.28  avg_score: 

episode:  411  score:  14.27  avg_score:  -33.42  frames:  138  epsilon:  0.44
episode:  412  score:  81.23  avg_score:  -32.37  frames:  999  epsilon:  0.44
episode:  413  score:  -95.71  avg_score:  -33.01  frames:  622  epsilon:  0.44
episode:  414  score:  -7.25  avg_score:  -32.91  frames:  129  epsilon:  0.44
episode:  415  score:  -80.77  avg_score:  -30.82  frames:  441  epsilon:  0.44
episode:  416  score:  -44.49  avg_score:  -31.07  frames:  70  epsilon:  0.44
episode:  417  score:  6.03  avg_score:  -30.67  frames:  117  epsilon:  0.43
episode:  418  score:  -22.14  avg_score:  -30.78  frames:  195  epsilon:  0.43
episode:  419  score:  17.77  avg_score:  -30.87  frames:  148  epsilon:  0.43
episode:  420  score:  -35.52  avg_score:  -30.85  frames:  179  epsilon:  0.43
episode:  421  score:  -27.51  avg_score:  -30.64  frames:  125  epsilon:  0.43
episode:  422  score:  10.30  avg_score:  -30.65  frames:  119  epsilon:  0.43
episode:  423  score:  -44.03  avg_score:  -30.7

...

episode:  829  score:  217.62  avg_score:  90.69  frames:  602  epsilon:  0.19
episode:  830  score:  132.36  avg_score:  91.40  frames:  999  epsilon:  0.19
episode:  831  score:  51.11  avg_score:  89.41  frames:  999  epsilon:  0.19
episode:  832  score:  210.49  avg_score:  92.56  frames:  490  epsilon:  0.19
episode:  833  score:  -218.76  avg_score:  87.70  frames:  226  epsilon:  0.19
episode:  834  score:  -16.30  avg_score:  85.43  frames:  999  epsilon:  0.19
episode:  835  score:  155.37  avg_score:  86.95  frames:  745  epsilon:  0.19
episode:  836  score:  200.39  avg_score:  89.31  frames:  475  epsilon:  0.19
episode:  837  score:  248.39  avg_score:  90.55  frames:  367  epsilon:  0.19
episode:  838  score:  215.50  avg_score:  92.63  frames:  760  epsilon:  0.19
episode:  839  score:  234.15  avg_score:  93.45  frames:  338  epsilon:  0.19
episode:  840  score:  229.14  avg_score:  93.09  frames:  821  epsilon:  0.19
episode:  841  score:  -52.66  avg_score:  91.59  fr

episode:  932  score:  216.19  avg_score:  144.24  frames:  918  epsilon:  0.16
episode:  933  score:  24.31  avg_score:  146.67  frames:  999  epsilon:  0.15
episode:  934  score:  250.77  avg_score:  149.34  frames:  797  epsilon:  0.15
episode:  935  score:  -70.05  avg_score:  147.09  frames:  91  epsilon:  0.15
episode:  936  score:  228.60  avg_score:  147.37  frames:  326  epsilon:  0.15
episode:  937  score:  233.94  avg_score:  147.23  frames:  502  epsilon:  0.15
episode:  938  score:  -22.19  avg_score:  144.85  frames:  240  epsilon:  0.15
episode:  939  score:  286.27  avg_score:  145.37  frames:  311  epsilon:  0.15
episode:  940  score:  -71.37  avg_score:  142.37  frames:  123  epsilon:  0.15
episode:  941  score:  238.93  avg_score:  145.28  frames:  260  epsilon:  0.15
episode:  942  score:  272.90  avg_score:  146.07  frames:  295  epsilon:  0.15
episode:  943  score:  224.25  avg_score:  147.31  frames:  215  epsilon:  0.15
episode:  944  score:  323.27  avg_score: 

episode:  1035  score:  235.43  avg_score:  145.81  frames:  696  epsilon:  0.13
episode:  1036  score:  163.83  avg_score:  145.16  frames:  861  epsilon:  0.13
episode:  1037  score:  251.66  avg_score:  145.34  frames:  310  epsilon:  0.13
episode:  1038  score:  233.35  avg_score:  147.89  frames:  316  epsilon:  0.13
episode:  1039  score:  257.72  avg_score:  147.61  frames:  286  epsilon:  0.13
episode:  1040  score:  1.87  avg_score:  148.34  frames:  117  epsilon:  0.13
episode:  1041  score:  234.64  avg_score:  148.30  frames:  239  epsilon:  0.12
episode:  1042  score:  293.26  avg_score:  148.50  frames:  332  epsilon:  0.12
episode:  1043  score:  20.32  avg_score:  146.46  frames:  85  epsilon:  0.12
episode:  1044  score:  286.81  avg_score:  146.09  frames:  307  epsilon:  0.12
episode:  1045  score:  272.81  avg_score:  146.12  frames:  250  epsilon:  0.12
episode:  1046  score:  274.37  avg_score:  146.46  frames:  394  epsilon:  0.12
episode:  1047  score:  242.49  

episode:  1137  score:  255.06  avg_score:  222.62  frames:  275  epsilon:  0.10
episode:  1138  score:  227.68  avg_score:  222.57  frames:  272  epsilon:  0.10
episode:  1139  score:  304.10  avg_score:  223.03  frames:  217  epsilon:  0.10
episode:  1140  score:  273.73  avg_score:  225.75  frames:  305  epsilon:  0.10
episode:  1141  score:  258.25  avg_score:  225.98  frames:  211  epsilon:  0.10
episode:  1142  score:  309.31  avg_score:  226.14  frames:  224  epsilon:  0.10
episode:  1143  score:  284.28  avg_score:  228.78  frames:  349  epsilon:  0.10
episode:  1144  score:  269.30  avg_score:  228.61  frames:  313  epsilon:  0.10
episode:  1145  score:  287.65  avg_score:  228.76  frames:  226  epsilon:  0.10
episode:  1146  score:  280.63  avg_score:  228.82  frames:  239  epsilon:  0.10
episode:  1147  score:  280.34  avg_score:  229.20  frames:  222  epsilon:  0.10
episode:  1148  score:  247.78  avg_score:  229.33  frames:  511  epsilon:  0.10
episode:  1149  score:  290.

KeyboardInterrupt: 

In [6]:
agent.model.save_weights('current_best.h5')

In [14]:
import gym, random, tempfile
import numpy as np
from gym import wrappers
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
agent.model.load_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                     +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

# Set to true to use saved model
episodes = 10000
env = gym.make('LunarLander-v2')
# Cumulative reward
reward_avg = [288.56,
 238.59,
 53.36,
 225.3,
 225.59,
 288.82,
 263.05,
 123.14,
 256.21,
 277.83,
 -4.46,
 289.93,
 249.57,
 15.07,
 289.92,
 280.19,
 -26.71,
 7.5,
 284.51,
 217.57,
 238.35,
 279.87,
 265.32,
 280.89,
 267.24,
 204.09,
 231.96,
 139.87,
 219.51,
 269.94,
 228.94,
 268.04,
 292.05,
 268.42,
 222.02,
 303.65,
 303.57,
 253.41,
 291.24,
 257.63,
 217.18,
 293.07,
 285.42,
 208.13,
 237.3,
 274.61,
 269.4,
 273.27,
 281.67,
 255.06,
 227.68,
 304.1,
 273.73,
 258.25,
 309.31,
 284.28,
 269.3,
 287.65,
 280.63,
 280.34,
 247.78,
 290.44,
 230.0,
 265.74,
 288.39,
 259.21,
 -26.51,
 295.55,
 262.62,
 -77.82,
 260.17,
 278.71,
 262.72,
 12.7,
 287.24,
 259.59,
 40.19,
 237.58,
 264.85,
 257.74,
 298.7,
 254.18,
 275.07,
 -21.37,
 257.33,
 39.36,
 276.33,
 291.36,
 253.3,
 228.58,
 288.1,
 119.16,
 271.24,
 229.94,
 219.42,
 231.31,
 247.61,
 254.04,
 11.39]

for e in range(1187,episodes):
    episode_reward = 0
    s = env.reset()
    s = np.reshape(s, [1, nS])

    for time in range(1000):

        # Query next action from learner and perform action
        a = agent.selectAction(s)
        s_prime, r, done, info = env.step(a)

        # Add cumulative reward
        episode_reward += r

        # Reshape new state
        s_prime = np.reshape(s_prime, [1, nS])

        # Add experience to memory
        agent.add_memory(s, a, r, s_prime, done)

        # Set current state to new state
        s = s_prime

        #Perform experience replay if memory length is greater than minibatch length

        if len(agent.memory) > agent.minibatch_size:
            agent.replay()

        # If episode is done, exit loop
        if done:
            agent.target_model_update()
            break


    # epsilon decay
    if agent.epsilon > agent.min_e:
        agent.epsilon *= agent.epsilon_decay
        agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

    # Running average of past 100 episodes
    reward_avg.append(episode_reward)
    print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg[-100:]), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)

    # with open('trained_agent.txt', 'a') as f:
    #     f.write(str(np.average(reward_avg)) + '\n')

env.close()


agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                     +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

episode:  1187  score:  -177.88  avg_score:  223.44  frames:  474  epsilon:  0.09
episode:  1188  score:  216.74  avg_score:  222.72  frames:  329  epsilon:  0.09
episode:  1189  score:  298.35  avg_score:  223.32  frames:  682  epsilon:  0.09
episode:  1190  score:  264.04  avg_score:  225.43  frames:  297  epsilon:  0.09
episode:  1191  score:  231.61  avg_score:  225.49  frames:  343  epsilon:  0.09
episode:  1192  score:  148.75  avg_score:  224.72  frames:  999  epsilon:  0.09
episode:  1193  score:  180.84  avg_score:  223.64  frames:  986  epsilon:  0.09
episode:  1194  score:  259.30  avg_score:  223.60  frames:  288  epsilon:  0.09
episode:  1195  score:  273.75  avg_score:  225.11  frames:  238  epsilon:  0.09
episode:  1196  score:  244.23  avg_score:  224.99  frames:  279  epsilon:  0.09
episode:  1197  score:  297.33  avg_score:  225.18  frames:  240  epsilon:  0.09
episode:  1198  score:  274.44  avg_score:  227.97  frames:  293  epsilon:  0.09
episode:  1199  score:  245

KeyboardInterrupt: 

In [18]:

for e in range(1212,episodes):
    episode_reward = 0
    s = env.reset()
    s = np.reshape(s, [1, nS])

    for time in range(1000):

        # Query next action from learner and perform action
        a = agent.selectAction(s)
        s_prime, r, done, info = env.step(a)

        # Add cumulative reward
        episode_reward += r

        # Reshape new state
        s_prime = np.reshape(s_prime, [1, nS])

        # Add experience to memory
        agent.add_memory(s, a, r, s_prime, done)

        # Set current state to new state
        s = s_prime

        #Perform experience replay if memory length is greater than minibatch length

        if len(agent.memory) > agent.minibatch_size:
            agent.replay()

        # If episode is done, exit loop
        if done:
            agent.target_model_update()
            break


    # epsilon decay
    if agent.epsilon > agent.min_e:
        agent.epsilon *= agent.epsilon_decay
        agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

    # Running average of past 100 episodes
    reward_avg.append(episode_reward)
    print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg[-100:]), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)

    # with open('trained_agent.txt', 'a') as f:
    #     f.write(str(np.average(reward_avg)) + '\n')

env.close()


agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                     +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

episode:  1212  score:  213.87  avg_score:  234.42  frames:  235  epsilon:  0.09
episode:  1213  score:  271.24  avg_score:  235.09  frames:  198  epsilon:  0.09
episode:  1214  score:  260.09  avg_score:  235.37  frames:  284  epsilon:  0.09
episode:  1215  score:  301.35  avg_score:  236.99  frames:  216  epsilon:  0.09
episode:  1216  score:  269.02  avg_score:  237.48  frames:  220  epsilon:  0.09
episode:  1217  score:  209.79  avg_score:  236.88  frames:  741  epsilon:  0.09
episode:  1218  score:  260.16  avg_score:  237.20  frames:  370  epsilon:  0.09


KeyboardInterrupt: 

In [15]:
agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')


In [16]:
'weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5'

'weights_DQN_1-0.0001-0.002-0.99-30-0.0002.h5'

In [19]:
string ="""
episode:  0  score:  -129.00  avg_score:  -129.00  frames:  96  epsilon:  1.00
episode:  1  score:  -166.53  avg_score:  -147.77  frames:  66  epsilon:  1.00
episode:  2  score:  -139.10  avg_score:  -144.88  frames:  86  epsilon:  1.00
episode:  3  score:  -67.02  avg_score:  -125.41  frames:  71  epsilon:  0.99
episode:  4  score:  -46.29  avg_score:  -109.59  frames:  59  epsilon:  0.99
episode:  5  score:  -120.50  avg_score:  -111.41  frames:  99  epsilon:  0.99
episode:  6  score:  -100.33  avg_score:  -109.83  frames:  78  epsilon:  0.99
episode:  7  score:  -128.64  avg_score:  -112.18  frames:  80  epsilon:  0.99
episode:  8  score:  -20.72  avg_score:  -102.02  frames:  92  epsilon:  0.98
episode:  9  score:  -86.17  avg_score:  -100.43  frames:  66  epsilon:  0.98
episode:  10  score:  -415.47  avg_score:  -129.07  frames:  108  epsilon:  0.98
episode:  11  score:  -192.95  avg_score:  -134.39  frames:  89  epsilon:  0.98
episode:  12  score:  -103.70  avg_score:  -132.03  frames:  71  epsilon:  0.98
episode:  13  score:  -277.32  avg_score:  -142.41  frames:  97  epsilon:  0.97
episode:  14  score:  -275.57  avg_score:  -151.29  frames:  78  epsilon:  0.97
episode:  15  score:  -130.91  avg_score:  -150.01  frames:  80  epsilon:  0.97
episode:  16  score:  -304.14  avg_score:  -159.08  frames:  132  epsilon:  0.97
episode:  17  score:  -112.50  avg_score:  -156.49  frames:  77  epsilon:  0.97
episode:  18  score:  -189.42  avg_score:  -158.23  frames:  115  epsilon:  0.96
episode:  19  score:  -164.54  avg_score:  -158.54  frames:  105  epsilon:  0.96
episode:  20  score:  -130.64  avg_score:  -157.21  frames:  77  epsilon:  0.96
episode:  21  score:  -224.55  avg_score:  -160.27  frames:  126  epsilon:  0.96
episode:  22  score:  -164.62  avg_score:  -160.46  frames:  79  epsilon:  0.96
episode:  23  score:  -172.85  avg_score:  -160.98  frames:  58  epsilon:  0.96
episode:  24  score:  -225.49  avg_score:  -163.56  frames:  94  epsilon:  0.95
episode:  25  score:  -18.25  avg_score:  -157.97  frames:  108  epsilon:  0.95
episode:  26  score:  -216.44  avg_score:  -160.14  frames:  108  epsilon:  0.95
episode:  27  score:  -304.44  avg_score:  -165.29  frames:  89  epsilon:  0.95
episode:  28  score:  -196.74  avg_score:  -166.37  frames:  88  epsilon:  0.95
episode:  29  score:  -105.07  avg_score:  -164.33  frames:  91  epsilon:  0.94
episode:  30  score:  -347.12  avg_score:  -170.23  frames:  93  epsilon:  0.94
episode:  31  score:  -94.37  avg_score:  -167.86  frames:  78  epsilon:  0.94
episode:  32  score:  -148.17  avg_score:  -167.26  frames:  75  epsilon:  0.94
episode:  33  score:  -52.80  avg_score:  -163.89  frames:  73  epsilon:  0.94
episode:  34  score:  -94.58  avg_score:  -161.91  frames:  76  epsilon:  0.93
episode:  35  score:  -344.25  avg_score:  -166.98  frames:  142  epsilon:  0.93
episode:  36  score:  -202.24  avg_score:  -167.93  frames:  87  epsilon:  0.93
episode:  37  score:  -96.14  avg_score:  -166.04  frames:  69  epsilon:  0.93
episode:  38  score:  -69.11  avg_score:  -163.56  frames:  100  epsilon:  0.93
episode:  39  score:  -251.74  avg_score:  -165.76  frames:  139  epsilon:  0.92
episode:  40  score:  -194.18  avg_score:  -166.45  frames:  117  epsilon:  0.92
episode:  41  score:  -85.74  avg_score:  -164.53  frames:  64  epsilon:  0.92
episode:  42  score:  -148.24  avg_score:  -164.15  frames:  137  epsilon:  0.92
episode:  43  score:  -157.62  avg_score:  -164.01  frames:  109  epsilon:  0.92
episode:  44  score:  -82.34  avg_score:  -162.19  frames:  78  epsilon:  0.92
episode:  45  score:  -185.67  avg_score:  -162.70  frames:  115  epsilon:  0.91
episode:  46  score:  -411.26  avg_score:  -167.99  frames:  102  epsilon:  0.91
episode:  47  score:  -408.61  avg_score:  -173.00  frames:  89  epsilon:  0.91
episode:  48  score:  -238.96  avg_score:  -174.35  frames:  84  epsilon:  0.91
episode:  49  score:  -84.40  avg_score:  -172.55  frames:  89  epsilon:  0.91
episode:  50  score:  -184.40  avg_score:  -172.78  frames:  96  epsilon:  0.90
episode:  51  score:  -268.29  avg_score:  -174.62  frames:  96  epsilon:  0.90
episode:  52  score:  -248.66  avg_score:  -176.02  frames:  103  epsilon:  0.90
episode:  53  score:  -424.53  avg_score:  -180.62  frames:  103  epsilon:  0.90
episode:  54  score:  -437.17  avg_score:  -185.28  frames:  90  epsilon:  0.90
episode:  55  score:  -141.04  avg_score:  -184.49  frames:  108  epsilon:  0.90
episode:  56  score:  -215.96  avg_score:  -185.04  frames:  80  epsilon:  0.89
episode:  57  score:  -226.37  avg_score:  -185.76  frames:  112  epsilon:  0.89
episode:  58  score:  -156.90  avg_score:  -185.27  frames:  71  epsilon:  0.89
episode:  59  score:  -115.63  avg_score:  -184.11  frames:  121  epsilon:  0.89
episode:  60  score:  -113.89  avg_score:  -182.96  frames:  68  epsilon:  0.89
episode:  61  score:  -159.55  avg_score:  -182.58  frames:  62  epsilon:  0.89
episode:  62  score:  -111.03  avg_score:  -181.44  frames:  101  epsilon:  0.88
episode:  63  score:  -71.81  avg_score:  -179.73  frames:  92  epsilon:  0.88
episode:  64  score:  -95.46  avg_score:  -178.43  frames:  74  epsilon:  0.88
episode:  65  score:  -53.73  avg_score:  -176.54  frames:  116  epsilon:  0.88
episode:  66  score:  -65.75  avg_score:  -174.89  frames:  88  epsilon:  0.88
episode:  67  score:  -84.50  avg_score:  -173.56  frames:  81  epsilon:  0.87
episode:  68  score:  -141.77  avg_score:  -173.10  frames:  96  epsilon:  0.87
episode:  69  score:  -20.46  avg_score:  -170.92  frames:  86  epsilon:  0.87
episode:  70  score:  -177.63  avg_score:  -171.01  frames:  68  epsilon:  0.87
episode:  71  score:  -245.73  avg_score:  -172.05  frames:  125  epsilon:  0.87
episode:  72  score:  -279.13  avg_score:  -173.52  frames:  113  epsilon:  0.87
episode:  73  score:  -72.61  avg_score:  -172.15  frames:  74  epsilon:  0.86
episode:  74  score:  -135.94  avg_score:  -171.67  frames:  107  epsilon:  0.86
episode:  75  score:  -284.81  avg_score:  -173.16  frames:  162  epsilon:  0.86
episode:  76  score:  -283.16  avg_score:  -174.59  frames:  123  epsilon:  0.86
episode:  77  score:  -86.36  avg_score:  -173.46  frames:  80  epsilon:  0.86
episode:  78  score:  -111.11  avg_score:  -172.67  frames:  112  epsilon:  0.86
episode:  79  score:  -142.04  avg_score:  -172.29  frames:  112  epsilon:  0.85
episode:  80  score:  -84.95  avg_score:  -171.21  frames:  100  epsilon:  0.85
episode:  81  score:  -101.27  avg_score:  -170.35  frames:  75  epsilon:  0.85
episode:  82  score:  -300.74  avg_score:  -171.93  frames:  92  epsilon:  0.85
episode:  83  score:  -5.03  avg_score:  -169.94  frames:  121  epsilon:  0.85
episode:  84  score:  -107.54  avg_score:  -169.20  frames:  71  epsilon:  0.85
episode:  85  score:  -239.15  avg_score:  -170.02  frames:  113  epsilon:  0.84
episode:  86  score:  8.36  avg_score:  -167.97  frames:  80  epsilon:  0.84
episode:  87  score:  -323.09  avg_score:  -169.73  frames:  135  epsilon:  0.84
episode:  88  score:  -183.66  avg_score:  -169.89  frames:  70  epsilon:  0.84
episode:  89  score:  -240.31  avg_score:  -170.67  frames:  87  epsilon:  0.84
episode:  90  score:  -121.04  avg_score:  -170.12  frames:  103  epsilon:  0.84
episode:  91  score:  -55.65  avg_score:  -168.88  frames:  68  epsilon:  0.83
episode:  92  score:  -97.77  avg_score:  -168.12  frames:  117  epsilon:  0.83
episode:  93  score:  -116.72  avg_score:  -167.57  frames:  88  epsilon:  0.83
episode:  94  score:  -109.12  avg_score:  -166.95  frames:  81  epsilon:  0.83
episode:  95  score:  -129.24  avg_score:  -166.56  frames:  90  epsilon:  0.83
episode:  96  score:  -191.78  avg_score:  -166.82  frames:  73  epsilon:  0.83
episode:  97  score:  -241.37  avg_score:  -167.58  frames:  114  epsilon:  0.82
episode:  98  score:  -83.72  avg_score:  -166.73  frames:  123  epsilon:  0.82
episode:  99  score:  -165.88  avg_score:  -166.73  frames:  97  epsilon:  0.82
episode:  100  score:  -131.60  avg_score:  -166.75  frames:  103  epsilon:  0.82
episode:  101  score:  -109.59  avg_score:  -166.18  frames:  121  epsilon:  0.82
episode:  102  score:  -221.50  avg_score:  -167.01  frames:  141  epsilon:  0.82
episode:  103  score:  -82.48  avg_score:  -167.16  frames:  71  epsilon:  0.81
episode:  104  score:  -146.75  avg_score:  -168.17  frames:  87  epsilon:  0.81
episode:  105  score:  -117.10  avg_score:  -168.13  frames:  144  epsilon:  0.81
episode:  106  score:  -106.12  avg_score:  -168.19  frames:  70  epsilon:  0.81
episode:  107  score:  -52.67  avg_score:  -167.43  frames:  147  epsilon:  0.81
episode:  108  score:  -66.55  avg_score:  -167.89  frames:  82  epsilon:  0.81
episode:  109  score:  -44.69  avg_score:  -167.47  frames:  87  epsilon:  0.80
episode:  110  score:  -145.66  avg_score:  -164.77  frames:  131  epsilon:  0.80
episode:  111  score:  -102.73  avg_score:  -163.87  frames:  96  epsilon:  0.80
episode:  112  score:  -148.38  avg_score:  -164.32  frames:  119  epsilon:  0.80
episode:  113  score:  -122.89  avg_score:  -162.78  frames:  102  epsilon:  0.80
episode:  114  score:  -129.17  avg_score:  -161.31  frames:  142  epsilon:  0.80
episode:  115  score:  -129.16  avg_score:  -161.29  frames:  107  epsilon:  0.79
episode:  116  score:  -131.74  avg_score:  -159.57  frames:  111  epsilon:  0.79
episode:  117  score:  -108.85  avg_score:  -159.53  frames:  128  epsilon:  0.79
episode:  118  score:  -93.44  avg_score:  -158.57  frames:  103  epsilon:  0.79
episode:  119  score:  -77.69  avg_score:  -157.71  frames:  151  epsilon:  0.79
episode:  120  score:  -16.46  avg_score:  -156.56  frames:  164  epsilon:  0.79
episode:  121  score:  -88.80  avg_score:  -155.21  frames:  116  epsilon:  0.79
episode:  122  score:  -101.00  avg_score:  -154.57  frames:  105  epsilon:  0.78
episode:  123  score:  -89.79  avg_score:  -153.74  frames:  66  epsilon:  0.78
episode:  124  score:  -121.28  avg_score:  -152.70  frames:  80  epsilon:  0.78
episode:  125  score:  -50.33  avg_score:  -153.02  frames:  90  epsilon:  0.78
episode:  126  score:  -34.79  avg_score:  -151.20  frames:  139  epsilon:  0.78
episode:  127  score:  -85.27  avg_score:  -149.01  frames:  113  epsilon:  0.78
episode:  128  score:  -43.58  avg_score:  -147.48  frames:  97  epsilon:  0.77
episode:  129  score:  -45.55  avg_score:  -146.88  frames:  85  epsilon:  0.77
episode:  130  score:  -160.84  avg_score:  -145.02  frames:  72  epsilon:  0.77
episode:  131  score:  0.40  avg_score:  -144.07  frames:  127  epsilon:  0.77
episode:  132  score:  -107.62  avg_score:  -143.67  frames:  132  epsilon:  0.77
episode:  133  score:  -89.36  avg_score:  -144.03  frames:  144  epsilon:  0.77
episode:  134  score:  -61.64  avg_score:  -143.70  frames:  80  epsilon:  0.76
episode:  135  score:  -13.37  avg_score:  -140.39  frames:  176  epsilon:  0.76
episode:  136  score:  -87.04  avg_score:  -139.24  frames:  101  epsilon:  0.76
episode:  137  score:  -123.46  avg_score:  -139.52  frames:  127  epsilon:  0.76
episode:  138  score:  -340.94  avg_score:  -142.23  frames:  124  epsilon:  0.76
episode:  139  score:  -157.23  avg_score:  -141.29  frames:  122  epsilon:  0.76
episode:  140  score:  -98.81  avg_score:  -140.33  frames:  88  epsilon:  0.76
episode:  141  score:  -93.82  avg_score:  -140.42  frames:  82  epsilon:  0.75
episode:  142  score:  -48.25  avg_score:  -139.42  frames:  113  epsilon:  0.75
episode:  143  score:  -77.45  avg_score:  -138.61  frames:  62  epsilon:  0.75
episode:  144  score:  -48.97  avg_score:  -138.28  frames:  88  epsilon:  0.75
episode:  145  score:  -76.21  avg_score:  -137.19  frames:  101  epsilon:  0.75
episode:  146  score:  -112.57  avg_score:  -134.20  frames:  113  epsilon:  0.75
episode:  147  score:  -73.27  avg_score:  -130.85  frames:  74  epsilon:  0.75
episode:  148  score:  -60.78  avg_score:  -129.06  frames:  91  epsilon:  0.74
episode:  149  score:  -40.34  avg_score:  -128.62  frames:  169  epsilon:  0.74
episode:  150  score:  -123.37  avg_score:  -128.01  frames:  135  epsilon:  0.74
episode:  151  score:  -116.14  avg_score:  -126.49  frames:  145  epsilon:  0.74
episode:  152  score:  -49.41  avg_score:  -124.50  frames:  133  epsilon:  0.74
episode:  153  score:  -70.71  avg_score:  -120.96  frames:  71  epsilon:  0.74
episode:  154  score:  -44.81  avg_score:  -117.04  frames:  83  epsilon:  0.73
episode:  155  score:  -97.36  avg_score:  -116.60  frames:  73  epsilon:  0.73
episode:  156  score:  -34.79  avg_score:  -114.79  frames:  107  epsilon:  0.73
episode:  157  score:  -130.59  avg_score:  -113.83  frames:  126  epsilon:  0.73
episode:  158  score:  -60.38  avg_score:  -112.87  frames:  80  epsilon:  0.73
episode:  159  score:  -2.67  avg_score:  -111.74  frames:  166  epsilon:  0.73
episode:  160  score:  -85.21  avg_score:  -111.45  frames:  104  epsilon:  0.73
episode:  161  score:  -105.84  avg_score:  -110.91  frames:  65  epsilon:  0.72
episode:  162  score:  -122.88  avg_score:  -111.03  frames:  111  epsilon:  0.72
episode:  163  score:  -58.48  avg_score:  -110.90  frames:  94  epsilon:  0.72
episode:  164  score:  -78.65  avg_score:  -110.73  frames:  120  epsilon:  0.72
episode:  165  score:  -64.29  avg_score:  -110.83  frames:  147  epsilon:  0.72
episode:  166  score:  -86.05  avg_score:  -111.04  frames:  86  epsilon:  0.72
episode:  167  score:  -118.63  avg_score:  -111.38  frames:  120  epsilon:  0.72
episode:  168  score:  -71.34  avg_score:  -110.67  frames:  155  epsilon:  0.71
episode:  169  score:  -61.09  avg_score:  -111.08  frames:  80  epsilon:  0.71
episode:  170  score:  -92.87  avg_score:  -110.23  frames:  119  epsilon:  0.71
episode:  171  score:  -75.67  avg_score:  -108.53  frames:  113  epsilon:  0.71
episode:  172  score:  -43.83  avg_score:  -106.18  frames:  76  epsilon:  0.71
episode:  173  score:  -86.08  avg_score:  -106.31  frames:  162  epsilon:  0.71
episode:  174  score:  -79.04  avg_score:  -105.75  frames:  90  epsilon:  0.71
episode:  175  score:  -69.79  avg_score:  -103.60  frames:  119  epsilon:  0.70
episode:  176  score:  -52.59  avg_score:  -101.29  frames:  167  epsilon:  0.70
episode:  177  score:  -66.92  avg_score:  -101.10  frames:  124  epsilon:  0.70
episode:  178  score:  -18.56  avg_score:  -100.17  frames:  160  epsilon:  0.70
episode:  179  score:  -31.99  avg_score:  -99.07  frames:  152  epsilon:  0.70
episode:  180  score:  -67.73  avg_score:  -98.90  frames:  190  epsilon:  0.70
episode:  181  score:  -111.72  avg_score:  -99.00  frames:  76  epsilon:  0.70
episode:  182  score:  -31.86  avg_score:  -96.31  frames:  150  epsilon:  0.69
episode:  183  score:  -45.25  avg_score:  -96.71  frames:  162  epsilon:  0.69
episode:  184  score:  -94.06  avg_score:  -96.58  frames:  152  epsilon:  0.69
episode:  185  score:  19.02  avg_score:  -94.00  frames:  145  epsilon:  0.69
episode:  186  score:  -91.96  avg_score:  -95.00  frames:  127  epsilon:  0.69
episode:  187  score:  -47.09  avg_score:  -92.24  frames:  172  epsilon:  0.69
episode:  188  score:  -134.77  avg_score:  -91.75  frames:  131  epsilon:  0.69
episode:  189  score:  -76.50  avg_score:  -90.11  frames:  82  epsilon:  0.69
episode:  190  score:  -76.95  avg_score:  -89.67  frames:  83  epsilon:  0.68
episode:  191  score:  -113.24  avg_score:  -90.25  frames:  76  epsilon:  0.68
episode:  192  score:  -86.06  avg_score:  -90.13  frames:  132  epsilon:  0.68
episode:  193  score:  -96.47  avg_score:  -89.93  frames:  107  epsilon:  0.68
episode:  194  score:  -129.70  avg_score:  -90.14  frames:  102  epsilon:  0.68
episode:  195  score:  -45.35  avg_score:  -89.30  frames:  127  epsilon:  0.68
episode:  196  score:  -100.61  avg_score:  -88.38  frames:  134  epsilon:  0.68
episode:  197  score:  -40.85  avg_score:  -86.38  frames:  199  epsilon:  0.67
episode:  198  score:  -82.78  avg_score:  -86.37  frames:  130  epsilon:  0.67
episode:  199  score:  -162.68  avg_score:  -86.34  frames:  119  epsilon:  0.67
episode:  200  score:  -102.56  avg_score:  -86.05  frames:  111  epsilon:  0.67
episode:  201  score:  -26.27  avg_score:  -85.21  frames:  109  epsilon:  0.67
episode:  202  score:  -56.22  avg_score:  -83.56  frames:  269  epsilon:  0.67
episode:  203  score:  -62.73  avg_score:  -83.36  frames:  118  epsilon:  0.67
episode:  204  score:  -130.99  avg_score:  -83.21  frames:  97  epsilon:  0.67
episode:  205  score:  -100.88  avg_score:  -83.04  frames:  122  epsilon:  0.66
episode:  206  score:  -81.06  avg_score:  -82.79  frames:  138  epsilon:  0.66
episode:  207  score:  -107.16  avg_score:  -83.34  frames:  64  epsilon:  0.66
episode:  208  score:  -1.05  avg_score:  -82.68  frames:  181  epsilon:  0.66
episode:  209  score:  -88.18  avg_score:  -83.12  frames:  119  epsilon:  0.66
episode:  210  score:  -11.10  avg_score:  -81.77  frames:  183  epsilon:  0.66
episode:  211  score:  -72.89  avg_score:  -81.47  frames:  105  epsilon:  0.66
episode:  212  score:  -100.41  avg_score:  -80.99  frames:  90  epsilon:  0.65
episode:  213  score:  -94.84  avg_score:  -80.71  frames:  104  epsilon:  0.65
episode:  214  score:  -46.90  avg_score:  -79.89  frames:  125  epsilon:  0.65
episode:  215  score:  -66.24  avg_score:  -79.26  frames:  150  epsilon:  0.65
episode:  216  score:  -7.54  avg_score:  -78.02  frames:  231  epsilon:  0.65
episode:  217  score:  -183.96  avg_score:  -78.77  frames:  529  epsilon:  0.65
episode:  218  score:  -9.39  avg_score:  -77.93  frames:  92  epsilon:  0.65
episode:  219  score:  -63.25  avg_score:  -77.79  frames:  101  epsilon:  0.65
episode:  220  score:  -216.63  avg_score:  -79.79  frames:  142  epsilon:  0.64
episode:  221  score:  -85.34  avg_score:  -79.75  frames:  176  epsilon:  0.64
episode:  222  score:  -48.38  avg_score:  -79.23  frames:  126  epsilon:  0.64
episode:  223  score:  -129.34  avg_score:  -79.62  frames:  125  epsilon:  0.64
episode:  224  score:  -127.04  avg_score:  -79.68  frames:  121  epsilon:  0.64
episode:  225  score:  -219.44  avg_score:  -81.37  frames:  207  epsilon:  0.64
episode:  226  score:  -45.46  avg_score:  -81.48  frames:  257  epsilon:  0.64
episode:  227  score:  -67.65  avg_score:  -81.30  frames:  94  epsilon:  0.64
episode:  228  score:  -75.35  avg_score:  -81.62  frames:  101  epsilon:  0.63
episode:  229  score:  -50.31  avg_score:  -81.67  frames:  122  epsilon:  0.63
episode:  230  score:  -107.45  avg_score:  -81.13  frames:  94  epsilon:  0.63
episode:  231  score:  -66.76  avg_score:  -81.80  frames:  176  epsilon:  0.63
episode:  232  score:  -34.00  avg_score:  -81.07  frames:  89  epsilon:  0.63
episode:  233  score:  -38.33  avg_score:  -80.56  frames:  155  epsilon:  0.63
episode:  234  score:  -55.18  avg_score:  -80.49  frames:  127  epsilon:  0.63
episode:  235  score:  -81.08  avg_score:  -81.17  frames:  108  epsilon:  0.63
episode:  236  score:  2.11  avg_score:  -80.28  frames:  83  epsilon:  0.62
episode:  237  score:  -112.22  avg_score:  -80.17  frames:  134  epsilon:  0.62
episode:  238  score:  23.95  avg_score:  -76.52  frames:  199  epsilon:  0.62
episode:  239  score:  32.08  avg_score:  -74.62  frames:  999  epsilon:  0.62
episode:  240  score:  -71.08  avg_score:  -74.35  frames:  100  epsilon:  0.62
episode:  241  score:  -172.72  avg_score:  -75.14  frames:  130  epsilon:  0.62
episode:  242  score:  -77.16  avg_score:  -75.43  frames:  166  epsilon:  0.62
episode:  243  score:  -54.15  avg_score:  -75.19  frames:  192  epsilon:  0.62
episode:  244  score:  -3.28  avg_score:  -74.74  frames:  167  epsilon:  0.61
episode:  245  score:  -57.71  avg_score:  -74.55  frames:  113  epsilon:  0.61
episode:  246  score:  -155.23  avg_score:  -74.98  frames:  150  epsilon:  0.61
episode:  247  score:  -18.81  avg_score:  -74.43  frames:  123  epsilon:  0.61
episode:  248  score:  39.99  avg_score:  -73.43  frames:  102  epsilon:  0.61
episode:  249  score:  -76.33  avg_score:  -73.79  frames:  69  epsilon:  0.61
episode:  250  score:  14.91  avg_score:  -72.40  frames:  999  epsilon:  0.61
episode:  251  score:  -64.70  avg_score:  -71.89  frames:  94  epsilon:  0.61
episode:  252  score:  -51.40  avg_score:  -71.91  frames:  139  epsilon:  0.60
episode:  253  score:  -51.48  avg_score:  -71.72  frames:  96  epsilon:  0.60
episode:  254  score:  -63.58  avg_score:  -71.90  frames:  74  epsilon:  0.60
episode:  255  score:  -28.85  avg_score:  -71.22  frames:  131  epsilon:  0.60
episode:  256  score:  -48.05  avg_score:  -71.35  frames:  179  epsilon:  0.60
episode:  257  score:  -43.15  avg_score:  -70.48  frames:  97  epsilon:  0.60
episode:  258  score:  -63.41  avg_score:  -70.51  frames:  178  epsilon:  0.60
episode:  259  score:  -34.05  avg_score:  -70.82  frames:  172  epsilon:  0.60
episode:  260  score:  -75.44  avg_score:  -70.72  frames:  141  epsilon:  0.59
episode:  261  score:  -65.06  avg_score:  -70.31  frames:  91  epsilon:  0.59
episode:  262  score:  -51.69  avg_score:  -69.60  frames:  149  epsilon:  0.59
episode:  263  score:  -55.61  avg_score:  -69.57  frames:  152  epsilon:  0.59
episode:  264  score:  -100.97  avg_score:  -69.80  frames:  97  epsilon:  0.59
episode:  265  score:  -42.51  avg_score:  -69.58  frames:  98  epsilon:  0.59
episode:  266  score:  -44.12  avg_score:  -69.16  frames:  148  epsilon:  0.59
episode:  267  score:  -39.23  avg_score:  -68.37  frames:  144  epsilon:  0.59
episode:  268  score:  -46.26  avg_score:  -68.12  frames:  158  epsilon:  0.59
episode:  269  score:  -20.15  avg_score:  -67.71  frames:  104  epsilon:  0.58
episode:  270  score:  73.22  avg_score:  -66.05  frames:  999  epsilon:  0.58
episode:  271  score:  -86.43  avg_score:  -66.15  frames:  143  epsilon:  0.58
episode:  272  score:  79.47  avg_score:  -64.92  frames:  999  epsilon:  0.58
episode:  273  score:  -8.65  avg_score:  -64.15  frames:  199  epsilon:  0.58
episode:  274  score:  -53.07  avg_score:  -63.89  frames:  115  epsilon:  0.58
episode:  275  score:  -65.45  avg_score:  -63.84  frames:  129  epsilon:  0.58
episode:  276  score:  -19.67  avg_score:  -63.51  frames:  176  epsilon:  0.58
episode:  277  score:  -23.08  avg_score:  -63.07  frames:  195  epsilon:  0.57
episode:  278  score:  -128.49  avg_score:  -64.17  frames:  138  epsilon:  0.57
episode:  279  score:  -61.94  avg_score:  -64.47  frames:  137  epsilon:  0.57
episode:  280  score:  1.15  avg_score:  -63.78  frames:  395  epsilon:  0.57
episode:  281  score:  -12.69  avg_score:  -62.79  frames:  137  epsilon:  0.57
episode:  282  score:  -29.10  avg_score:  -62.77  frames:  109  epsilon:  0.57
episode:  283  score:  -2.92  avg_score:  -62.34  frames:  169  epsilon:  0.57
episode:  284  score:  -21.57  avg_score:  -61.62  frames:  173  epsilon:  0.57
episode:  285  score:  -98.35  avg_score:  -62.79  frames:  103  epsilon:  0.57
episode:  286  score:  -106.92  avg_score:  -62.94  frames:  119  epsilon:  0.56
episode:  287  score:  -127.86  avg_score:  -63.75  frames:  174  epsilon:  0.56
episode:  288  score:  1.61  avg_score:  -62.39  frames:  143  epsilon:  0.56
episode:  289  score:  -19.01  avg_score:  -61.81  frames:  166  epsilon:  0.56
episode:  290  score:  5.75  avg_score:  -60.98  frames:  179  epsilon:  0.56
episode:  291  score:  -40.10  avg_score:  -60.25  frames:  135  epsilon:  0.56
episode:  292  score:  -45.85  avg_score:  -59.85  frames:  99  epsilon:  0.56
episode:  293  score:  4.20  avg_score:  -58.84  frames:  170  epsilon:  0.56
episode:  294  score:  -18.29  avg_score:  -57.73  frames:  202  epsilon:  0.56
episode:  295  score:  -10.58  avg_score:  -57.38  frames:  283  epsilon:  0.55
episode:  296  score:  -79.53  avg_score:  -57.17  frames:  186  epsilon:  0.55
episode:  297  score:  -22.16  avg_score:  -56.98  frames:  129  epsilon:  0.55
episode:  298  score:  -31.18  avg_score:  -56.47  frames:  123  epsilon:  0.55
episode:  299  score:  -45.52  avg_score:  -55.30  frames:  136  epsilon:  0.55
episode:  300  score:  -44.17  avg_score:  -54.71  frames:  230  epsilon:  0.55
episode:  301  score:  -14.84  avg_score:  -54.60  frames:  117  epsilon:  0.55
episode:  302  score:  -28.15  avg_score:  -54.32  frames:  114  epsilon:  0.55
episode:  303  score:  13.87  avg_score:  -53.55  frames:  112  epsilon:  0.55
episode:  304  score:  -11.76  avg_score:  -52.36  frames:  113  epsilon:  0.54
episode:  305  score:  -48.61  avg_score:  -51.84  frames:  106  epsilon:  0.54
episode:  306  score:  -13.46  avg_score:  -51.16  frames:  185  epsilon:  0.54
episode:  307  score:  1.51  avg_score:  -50.07  frames:  201  epsilon:  0.54
episode:  308  score:  -49.64  avg_score:  -50.56  frames:  167  epsilon:  0.54
episode:  309  score:  -6.24  avg_score:  -49.74  frames:  202  epsilon:  0.54
episode:  310  score:  -28.37  avg_score:  -49.91  frames:  116  epsilon:  0.54
episode:  311  score:  21.72  avg_score:  -48.97  frames:  348  epsilon:  0.54
episode:  312  score:  -23.99  avg_score:  -48.20  frames:  158  epsilon:  0.54
episode:  313  score:  -31.63  avg_score:  -47.57  frames:  128  epsilon:  0.53
episode:  314  score:  -16.88  avg_score:  -47.27  frames:  107  epsilon:  0.53
episode:  315  score:  -290.47  avg_score:  -49.51  frames:  589  epsilon:  0.53
episode:  316  score:  -18.99  avg_score:  -49.63  frames:  999  epsilon:  0.53
episode:  317  score:  -34.47  avg_score:  -48.13  frames:  129  epsilon:  0.53
episode:  318  score:  -10.88  avg_score:  -48.15  frames:  119  epsilon:  0.53
episode:  319  score:  26.59  avg_score:  -47.25  frames:  168  epsilon:  0.53
episode:  320  score:  -37.28  avg_score:  -45.46  frames:  136  epsilon:  0.53
episode:  321  score:  -48.62  avg_score:  -45.09  frames:  77  epsilon:  0.53
episode:  322  score:  10.98  avg_score:  -44.50  frames:  999  epsilon:  0.53
episode:  323  score:  -38.55  avg_score:  -43.59  frames:  85  epsilon:  0.52
episode:  324  score:  0.37  avg_score:  -42.31  frames:  104  epsilon:  0.52
episode:  325  score:  -54.87  avg_score:  -40.67  frames:  85  epsilon:  0.52
episode:  326  score:  -25.56  avg_score:  -40.47  frames:  117  epsilon:  0.52
episode:  327  score:  -0.11  avg_score:  -39.79  frames:  142  epsilon:  0.52
episode:  328  score:  -34.60  avg_score:  -39.39  frames:  157  epsilon:  0.52
episode:  329  score:  -6.96  avg_score:  -38.95  frames:  103  epsilon:  0.52
episode:  330  score:  -29.11  avg_score:  -38.17  frames:  108  epsilon:  0.52
episode:  331  score:  -41.11  avg_score:  -37.91  frames:  129  epsilon:  0.52
episode:  332  score:  -42.83  avg_score:  -38.00  frames:  84  epsilon:  0.51
episode:  333  score:  -7.15  avg_score:  -37.69  frames:  98  epsilon:  0.51
episode:  334  score:  -160.62  avg_score:  -38.74  frames:  136  epsilon:  0.51
episode:  335  score:  -13.18  avg_score:  -38.06  frames:  123  epsilon:  0.51
episode:  336  score:  -50.02  avg_score:  -38.59  frames:  124  epsilon:  0.51
episode:  337  score:  26.36  avg_score:  -37.20  frames:  115  epsilon:  0.51
episode:  338  score:  75.76  avg_score:  -36.68  frames:  999  epsilon:  0.51
episode:  339  score:  -76.00  avg_score:  -37.76  frames:  124  epsilon:  0.51
episode:  340  score:  25.32  avg_score:  -36.80  frames:  148  epsilon:  0.51
episode:  341  score:  67.60  avg_score:  -34.40  frames:  999  epsilon:  0.51
episode:  342  score:  -55.80  avg_score:  -34.18  frames:  99  epsilon:  0.50
episode:  343  score:  -23.26  avg_score:  -33.87  frames:  151  epsilon:  0.50
episode:  344  score:  -11.37  avg_score:  -33.95  frames:  209  epsilon:  0.50
episode:  345  score:  -3.13  avg_score:  -33.41  frames:  111  epsilon:  0.50
episode:  346  score:  -13.84  avg_score:  -31.99  frames:  110  epsilon:  0.50
episode:  347  score:  -226.05  avg_score:  -34.07  frames:  138  epsilon:  0.50
episode:  348  score:  14.06  avg_score:  -34.33  frames:  163  epsilon:  0.50
episode:  349  score:  -45.60  avg_score:  -34.02  frames:  96  epsilon:  0.50
episode:  350  score:  -137.72  avg_score:  -35.54  frames:  430  epsilon:  0.50
episode:  351  score:  -110.64  avg_score:  -36.00  frames:  332  epsilon:  0.50
episode:  352  score:  -14.78  avg_score:  -35.64  frames:  176  epsilon:  0.49
episode:  353  score:  -87.18  avg_score:  -35.99  frames:  452  epsilon:  0.49
episode:  354  score:  -35.73  avg_score:  -35.72  frames:  101  epsilon:  0.49
episode:  355  score:  -43.38  avg_score:  -35.86  frames:  170  epsilon:  0.49
episode:  356  score:  -22.88  avg_score:  -35.61  frames:  125  epsilon:  0.49
episode:  357  score:  -12.48  avg_score:  -35.30  frames:  165  epsilon:  0.49
episode:  358  score:  -26.37  avg_score:  -34.93  frames:  141  epsilon:  0.49
episode:  359  score:  16.81  avg_score:  -34.42  frames:  181  epsilon:  0.49
episode:  360  score:  35.96  avg_score:  -33.31  frames:  112  epsilon:  0.49
episode:  361  score:  70.58  avg_score:  -31.95  frames:  999  epsilon:  0.49
episode:  362  score:  -23.98  avg_score:  -31.68  frames:  999  epsilon:  0.48
episode:  363  score:  -10.33  avg_score:  -31.22  frames:  117  epsilon:  0.48
episode:  364  score:  13.97  avg_score:  -30.07  frames:  999  epsilon:  0.48
episode:  365  score:  -93.86  avg_score:  -30.59  frames:  108  epsilon:  0.48
episode:  366  score:  3.66  avg_score:  -30.11  frames:  143  epsilon:  0.48
episode:  367  score:  -10.07  avg_score:  -29.82  frames:  98  epsilon:  0.48
episode:  368  score:  -21.29  avg_score:  -29.57  frames:  110  epsilon:  0.48
episode:  369  score:  -228.43  avg_score:  -31.65  frames:  134  epsilon:  0.48
episode:  370  score:  4.67  avg_score:  -32.34  frames:  166  epsilon:  0.48
episode:  371  score:  -22.79  avg_score:  -31.70  frames:  135  epsilon:  0.48
episode:  372  score:  39.29  avg_score:  -32.10  frames:  131  epsilon:  0.48
episode:  373  score:  -71.59  avg_score:  -32.73  frames:  129  epsilon:  0.47
episode:  374  score:  -44.73  avg_score:  -32.65  frames:  156  epsilon:  0.47
episode:  375  score:  -34.63  avg_score:  -32.34  frames:  109  epsilon:  0.47
episode:  376  score:  -36.12  avg_score:  -32.51  frames:  135  epsilon:  0.47
episode:  377  score:  62.95  avg_score:  -31.64  frames:  134  epsilon:  0.47
episode:  378  score:  -79.36  avg_score:  -31.15  frames:  152  epsilon:  0.47
episode:  379  score:  -32.12  avg_score:  -30.86  frames:  130  epsilon:  0.47
episode:  380  score:  24.29  avg_score:  -30.62  frames:  120  epsilon:  0.47
episode:  381  score:  2.68  avg_score:  -30.47  frames:  106  epsilon:  0.47
episode:  382  score:  -12.89  avg_score:  -30.31  frames:  125  epsilon:  0.47
episode:  383  score:  -35.62  avg_score:  -30.64  frames:  119  epsilon:  0.46
episode:  384  score:  -144.78  avg_score:  -31.87  frames:  519  epsilon:  0.46
episode:  385  score:  17.12  avg_score:  -30.71  frames:  237  epsilon:  0.46
episode:  386  score:  21.06  avg_score:  -29.43  frames:  120  epsilon:  0.46
episode:  387  score:  20.10  avg_score:  -27.95  frames:  999  epsilon:  0.46
episode:  388  score:  -179.88  avg_score:  -29.77  frames:  332  epsilon:  0.46
episode:  389  score:  -199.01  avg_score:  -31.57  frames:  592  epsilon:  0.46
episode:  390  score:  29.03  avg_score:  -31.34  frames:  218  epsilon:  0.46
episode:  391  score:  -45.80  avg_score:  -31.39  frames:  119  epsilon:  0.46
episode:  392  score:  53.35  avg_score:  -30.40  frames:  999  epsilon:  0.46
episode:  393  score:  -34.86  avg_score:  -30.79  frames:  84  epsilon:  0.46
episode:  394  score:  -12.93  avg_score:  -30.74  frames:  136  epsilon:  0.45
episode:  395  score:  -324.15  avg_score:  -33.87  frames:  969  epsilon:  0.45
episode:  396  score:  -91.66  avg_score:  -33.99  frames:  245  epsilon:  0.45
episode:  397  score:  16.60  avg_score:  -33.61  frames:  999  epsilon:  0.45
episode:  398  score:  -21.28  avg_score:  -33.51  frames:  180  epsilon:  0.45
episode:  399  score:  -33.95  avg_score:  -33.39  frames:  999  epsilon:  0.45
episode:  400  score:  30.46  avg_score:  -32.65  frames:  999  epsilon:  0.45
episode:  401  score:  -47.63  avg_score:  -32.97  frames:  141  epsilon:  0.45
episode:  402  score:  -77.11  avg_score:  -33.46  frames:  134  epsilon:  0.45
episode:  403  score:  -11.92  avg_score:  -33.72  frames:  144  epsilon:  0.45
episode:  404  score:  72.10  avg_score:  -32.88  frames:  999  epsilon:  0.45
episode:  405  score:  -43.21  avg_score:  -32.83  frames:  87  epsilon:  0.44
episode:  406  score:  -43.72  avg_score:  -33.13  frames:  999  epsilon:  0.44
episode:  407  score:  -14.69  avg_score:  -33.29  frames:  169  epsilon:  0.44
episode:  408  score:  -13.41  avg_score:  -32.93  frames:  999  epsilon:  0.44
episode:  409  score:  1.78  avg_score:  -32.85  frames:  100  epsilon:  0.44
episode:  410  score:  -78.14  avg_score:  -33.35  frames:  129  epsilon:  0.44
episode:  411  score:  14.27  avg_score:  -33.42  frames:  138  epsilon:  0.44
episode:  412  score:  81.23  avg_score:  -32.37  frames:  999  epsilon:  0.44
episode:  413  score:  -95.71  avg_score:  -33.01  frames:  622  epsilon:  0.44
episode:  414  score:  -7.25  avg_score:  -32.91  frames:  129  epsilon:  0.44
episode:  415  score:  -80.77  avg_score:  -30.82  frames:  441  epsilon:  0.44
episode:  416  score:  -44.49  avg_score:  -31.07  frames:  70  epsilon:  0.44
episode:  417  score:  6.03  avg_score:  -30.67  frames:  117  epsilon:  0.43
episode:  418  score:  -22.14  avg_score:  -30.78  frames:  195  epsilon:  0.43
episode:  419  score:  17.77  avg_score:  -30.87  frames:  148  epsilon:  0.43
episode:  420  score:  -35.52  avg_score:  -30.85  frames:  179  epsilon:  0.43
episode:  421  score:  -27.51  avg_score:  -30.64  frames:  125  epsilon:  0.43
episode:  422  score:  10.30  avg_score:  -30.65  frames:  119  epsilon:  0.43
episode:  423  score:  -44.03  avg_score:  -30.70  frames:  114  epsilon:  0.43
episode:  424  score:  -49.08  avg_score:  -31.20  frames:  110  epsilon:  0.43
episode:  425  score:  -22.81  avg_score:  -30.88  frames:  999  epsilon:  0.43
episode:  426  score:  68.57  avg_score:  -29.93  frames:  999  epsilon:  0.43
episode:  427  score:  38.47  avg_score:  -29.55  frames:  169  epsilon:  0.43
episode:  428  score:  -70.65  avg_score:  -29.91  frames:  279  epsilon:  0.42
episode:  429  score:  -81.28  avg_score:  -30.65  frames:  64  epsilon:  0.42
episode:  430  score:  -37.08  avg_score:  -30.73  frames:  101  epsilon:  0.42
episode:  431  score:  -42.96  avg_score:  -30.75  frames:  135  epsilon:  0.42
episode:  432  score:  30.36  avg_score:  -30.02  frames:  137  epsilon:  0.42
episode:  433  score:  80.08  avg_score:  -29.15  frames:  999  epsilon:  0.42
episode:  434  score:  -244.76  avg_score:  -29.99  frames:  401  epsilon:  0.42
episode:  435  score:  -107.63  avg_score:  -30.93  frames:  82  epsilon:  0.42
episode:  436  score:  -257.32  avg_score:  -33.01  frames:  386  epsilon:  0.42
episode:  437  score:  -12.11  avg_score:  -33.39  frames:  999  epsilon:  0.42
episode:  438  score:  -56.55  avg_score:  -34.71  frames:  125  epsilon:  0.42
episode:  439  score:  61.06  avg_score:  -33.34  frames:  999  epsilon:  0.42
episode:  440  score:  -18.37  avg_score:  -33.78  frames:  999  epsilon:  0.41
episode:  441  score:  -0.59  avg_score:  -34.46  frames:  341  epsilon:  0.41
episode:  442  score:  -21.41  avg_score:  -34.12  frames:  999  epsilon:  0.41
episode:  443  score:  9.07  avg_score:  -33.79  frames:  999  epsilon:  0.41
episode:  444  score:  -10.56  avg_score:  -33.79  frames:  117  epsilon:  0.41
episode:  445  score:  3.76  avg_score:  -33.72  frames:  131  epsilon:  0.41
episode:  446  score:  -41.70  avg_score:  -34.00  frames:  113  epsilon:  0.41
episode:  447  score:  -26.32  avg_score:  -32.00  frames:  133  epsilon:  0.41
episode:  448  score:  34.69  avg_score:  -31.79  frames:  999  epsilon:  0.41
episode:  449  score:  -60.21  avg_score:  -31.94  frames:  149  epsilon:  0.41
episode:  450  score:  76.62  avg_score:  -29.79  frames:  999  epsilon:  0.41
episode:  451  score:  63.59  avg_score:  -28.05  frames:  999  epsilon:  0.41
episode:  452  score:  -137.15  avg_score:  -29.28  frames:  116  epsilon:  0.41
episode:  453  score:  19.24  avg_score:  -28.21  frames:  115  epsilon:  0.40
episode:  454  score:  63.94  avg_score:  -27.21  frames:  999  epsilon:  0.40
episode:  455  score:  -62.67  avg_score:  -27.41  frames:  267  epsilon:  0.40
episode:  456  score:  9.21  avg_score:  -27.09  frames:  999  epsilon:  0.40
episode:  457  score:  -11.99  avg_score:  -27.08  frames:  114  epsilon:  0.40
episode:  458  score:  86.84  avg_score:  -25.95  frames:  999  epsilon:  0.40
episode:  459  score:  108.97  avg_score:  -25.03  frames:  999  epsilon:  0.40
episode:  460  score:  -25.83  avg_score:  -25.65  frames:  105  epsilon:  0.40
episode:  461  score:  73.98  avg_score:  -25.61  frames:  999  epsilon:  0.40
episode:  462  score:  -13.94  avg_score:  -25.51  frames:  91  epsilon:  0.40
episode:  463  score:  0.50  avg_score:  -25.40  frames:  215  epsilon:  0.40
episode:  464  score:  109.16  avg_score:  -24.45  frames:  999  epsilon:  0.40
episode:  465  score:  13.71  avg_score:  -23.38  frames:  126  epsilon:  0.39
episode:  466  score:  58.70  avg_score:  -22.83  frames:  999  epsilon:  0.39
episode:  467  score:  -49.50  avg_score:  -23.22  frames:  109  epsilon:  0.39
episode:  468  score:  -27.84  avg_score:  -23.29  frames:  101  epsilon:  0.39
episode:  469  score:  -44.59  avg_score:  -21.45  frames:  93  epsilon:  0.39
episode:  470  score:  29.03  avg_score:  -21.20  frames:  999  epsilon:  0.39
episode:  471  score:  -37.65  avg_score:  -21.35  frames:  112  epsilon:  0.39
episode:  472  score:  8.60  avg_score:  -21.66  frames:  186  epsilon:  0.39
episode:  473  score:  -8.24  avg_score:  -21.03  frames:  157  epsilon:  0.39
episode:  474  score:  92.18  avg_score:  -19.66  frames:  999  epsilon:  0.39
episode:  475  score:  98.98  avg_score:  -18.32  frames:  999  epsilon:  0.39
episode:  476  score:  69.14  avg_score:  -17.27  frames:  999  epsilon:  0.39
episode:  477  score:  88.05  avg_score:  -17.02  frames:  999  epsilon:  0.39
episode:  478  score:  -24.52  avg_score:  -16.47  frames:  80  epsilon:  0.38
episode:  479  score:  -0.02  avg_score:  -16.15  frames:  330  epsilon:  0.38
episode:  480  score:  1.23  avg_score:  -16.38  frames:  158  epsilon:  0.38
episode:  481  score:  -13.80  avg_score:  -16.54  frames:  99  epsilon:  0.38
episode:  482  score:  -16.28  avg_score:  -16.58  frames:  118  epsilon:  0.38
episode:  483  score:  115.29  avg_score:  -15.07  frames:  999  epsilon:  0.38
episode:  484  score:  -8.15  avg_score:  -13.70  frames:  117  epsilon:  0.38
episode:  485  score:  -47.19  avg_score:  -14.34  frames:  98  epsilon:  0.38
episode:  486  score:  -34.96  avg_score:  -14.90  frames:  108  epsilon:  0.38
episode:  487  score:  73.90  avg_score:  -14.37  frames:  999  epsilon:  0.38
episode:  488  score:  10.64  avg_score:  -12.46  frames:  141  epsilon:  0.38
episode:  489  score:  -99.21  avg_score:  -11.46  frames:  113  epsilon:  0.38
episode:  490  score:  9.03  avg_score:  -11.66  frames:  161  epsilon:  0.38
episode:  491  score:  -150.95  avg_score:  -12.71  frames:  141  epsilon:  0.37
episode:  492  score:  8.13  avg_score:  -13.17  frames:  178  epsilon:  0.37
episode:  493  score:  13.69  avg_score:  -12.68  frames:  124  epsilon:  0.37
episode:  494  score:  100.76  avg_score:  -11.54  frames:  999  epsilon:  0.37
episode:  495  score:  -21.87  avg_score:  -8.52  frames:  107  epsilon:  0.37
episode:  496  score:  -57.45  avg_score:  -8.18  frames:  109  epsilon:  0.37
episode:  497  score:  -14.61  avg_score:  -8.49  frames:  432  epsilon:  0.37
episode:  498  score:  -8.40  avg_score:  -8.36  frames:  143  epsilon:  0.37
episode:  499  score:  34.25  avg_score:  -7.68  frames:  151  epsilon:  0.37
episode:  500  score:  -87.21  avg_score:  -8.86  frames:  97  epsilon:  0.37
episode:  501  score:  108.54  avg_score:  -7.30  frames:  999  epsilon:  0.37
episode:  502  score:  -21.83  avg_score:  -6.74  frames:  88  epsilon:  0.37
episode:  503  score:  -2.64  avg_score:  -6.65  frames:  120  epsilon:  0.37
episode:  504  score:  -8.48  avg_score:  -7.46  frames:  110  epsilon:  0.37
episode:  505  score:  -25.78  avg_score:  -7.28  frames:  163  epsilon:  0.36
episode:  506  score:  -36.12  avg_score:  -7.21  frames:  487  epsilon:  0.36
episode:  507  score:  38.29  avg_score:  -6.68  frames:  999  epsilon:  0.36
episode:  508  score:  -174.26  avg_score:  -8.28  frames:  260  epsilon:  0.36
episode:  509  score:  -87.06  avg_score:  -9.17  frames:  211  epsilon:  0.36
episode:  510  score:  12.17  avg_score:  -8.27  frames:  183  epsilon:  0.36
episode:  511  score:  -83.83  avg_score:  -9.25  frames:  999  epsilon:  0.36
episode:  512  score:  47.90  avg_score:  -9.58  frames:  91  epsilon:  0.36
episode:  513  score:  267.24  avg_score:  -5.95  frames:  335  epsilon:  0.36
episode:  514  score:  -201.35  avg_score:  -7.90  frames:  253  epsilon:  0.36
episode:  515  score:  -63.12  avg_score:  -7.72  frames:  180  epsilon:  0.36
episode:  516  score:  -254.10  avg_score:  -9.81  frames:  238  epsilon:  0.36
episode:  517  score:  118.76  avg_score:  -8.69  frames:  999  epsilon:  0.36
episode:  518  score:  67.93  avg_score:  -7.79  frames:  999  epsilon:  0.35
episode:  519  score:  121.97  avg_score:  -6.75  frames:  999  epsilon:  0.35
episode:  520  score:  87.14  avg_score:  -5.52  frames:  999  epsilon:  0.35
episode:  521  score:  228.77  avg_score:  -2.96  frames:  345  epsilon:  0.35
episode:  522  score:  -32.26  avg_score:  -3.38  frames:  122  epsilon:  0.35
episode:  523  score:  -38.75  avg_score:  -3.33  frames:  69  epsilon:  0.35
episode:  524  score:  74.01  avg_score:  -2.10  frames:  999  epsilon:  0.35
episode:  525  score:  122.78  avg_score:  -0.64  frames:  999  epsilon:  0.35
episode:  526  score:  11.51  avg_score:  -1.21  frames:  284  epsilon:  0.35
episode:  527  score:  13.69  avg_score:  -1.46  frames:  999  epsilon:  0.35
episode:  528  score:  -21.83  avg_score:  -0.97  frames:  78  epsilon:  0.35
episode:  529  score:  85.90  avg_score:  0.70  frames:  999  epsilon:  0.35
episode:  530  score:  90.67  avg_score:  1.98  frames:  999  epsilon:  0.35
episode:  531  score:  94.36  avg_score:  3.35  frames:  999  epsilon:  0.35
episode:  532  score:  113.40  avg_score:  4.18  frames:  999  epsilon:  0.35
episode:  533  score:  81.13  avg_score:  4.19  frames:  999  epsilon:  0.34
episode:  534  score:  67.57  avg_score:  7.31  frames:  999  epsilon:  0.34
episode:  535  score:  16.76  avg_score:  8.56  frames:  129  epsilon:  0.34
episode:  536  score:  168.19  avg_score:  12.81  frames:  999  epsilon:  0.34
episode:  537  score:  18.81  avg_score:  13.12  frames:  999  epsilon:  0.34
episode:  538  score:  8.91  avg_score:  13.78  frames:  144  epsilon:  0.34
episode:  539  score:  86.34  avg_score:  14.03  frames:  999  epsilon:  0.34
episode:  540  score:  0.52  avg_score:  14.22  frames:  999  epsilon:  0.34
episode:  541  score:  0.52  avg_score:  14.23  frames:  76  epsilon:  0.34
episode:  542  score:  115.81  avg_score:  15.60  frames:  999  epsilon:  0.34
episode:  543  score:  120.77  avg_score:  16.72  frames:  999  epsilon:  0.34
episode:  544  score:  244.87  avg_score:  19.27  frames:  406  epsilon:  0.34
episode:  545  score:  192.08  avg_score:  21.16  frames:  777  epsilon:  0.34
episode:  546  score:  -18.65  avg_score:  21.39  frames:  132  epsilon:  0.34
episode:  547  score:  26.95  avg_score:  21.92  frames:  999  epsilon:  0.33
episode:  548  score:  124.47  avg_score:  22.82  frames:  999  epsilon:  0.33
episode:  549  score:  172.75  avg_score:  25.15  frames:  999  epsilon:  0.33
episode:  550  score:  232.52  avg_score:  26.71  frames:  756  epsilon:  0.33
episode:  551  score:  -71.85  avg_score:  25.35  frames:  107  epsilon:  0.33
episode:  552  score:  10.79  avg_score:  26.83  frames:  119  epsilon:  0.33
episode:  553  score:  91.60  avg_score:  27.56  frames:  999  epsilon:  0.33
episode:  554  score:  61.32  avg_score:  27.53  frames:  999  epsilon:  0.33
episode:  555  score:  120.70  avg_score:  29.36  frames:  999  epsilon:  0.33
episode:  556  score:  -437.96  avg_score:  24.89  frames:  339  epsilon:  0.33
episode:  557  score:  85.86  avg_score:  25.87  frames:  999  epsilon:  0.33
episode:  558  score:  13.57  avg_score:  25.14  frames:  137  epsilon:  0.33
episode:  559  score:  20.22  avg_score:  24.25  frames:  135  epsilon:  0.33
episode:  560  score:  13.43  avg_score:  24.64  frames:  111  epsilon:  0.33
episode:  561  score:  153.01  avg_score:  25.43  frames:  999  epsilon:  0.33
episode:  562  score:  34.88  avg_score:  25.92  frames:  999  epsilon:  0.33
episode:  563  score:  65.39  avg_score:  26.57  frames:  999  epsilon:  0.32
episode:  564  score:  127.25  avg_score:  26.75  frames:  999  epsilon:  0.32
episode:  565  score:  78.96  avg_score:  27.40  frames:  999  epsilon:  0.32
episode:  566  score:  -71.67  avg_score:  26.10  frames:  260  epsilon:  0.32
episode:  567  score:  8.41  avg_score:  26.68  frames:  122  epsilon:  0.32
episode:  568  score:  252.58  avg_score:  29.48  frames:  750  epsilon:  0.32
episode:  569  score:  48.85  avg_score:  30.42  frames:  270  epsilon:  0.32
episode:  570  score:  -197.88  avg_score:  28.15  frames:  336  epsilon:  0.32
episode:  571  score:  147.72  avg_score:  30.00  frames:  999  epsilon:  0.32
episode:  572  score:  61.69  avg_score:  30.53  frames:  999  epsilon:  0.32
episode:  573  score:  158.67  avg_score:  32.20  frames:  999  epsilon:  0.32
episode:  574  score:  31.15  avg_score:  31.59  frames:  186  epsilon:  0.32
episode:  575  score:  94.40  avg_score:  31.54  frames:  999  epsilon:  0.32
episode:  576  score:  125.54  avg_score:  32.11  frames:  999  epsilon:  0.32
episode:  577  score:  -316.03  avg_score:  28.07  frames:  223  epsilon:  0.32
episode:  578  score:  154.45  avg_score:  29.86  frames:  999  epsilon:  0.31
episode:  579  score:  78.23  avg_score:  30.64  frames:  999  epsilon:  0.31
episode:  580  score:  -31.73  avg_score:  30.31  frames:  124  epsilon:  0.31
episode:  581  score:  113.55  avg_score:  31.58  frames:  999  epsilon:  0.31
episode:  582  score:  115.64  avg_score:  32.90  frames:  999  epsilon:  0.31
episode:  583  score:  272.54  avg_score:  34.48  frames:  846  epsilon:  0.31
episode:  584  score:  92.21  avg_score:  35.48  frames:  999  epsilon:  0.31
episode:  585  score:  25.09  avg_score:  36.20  frames:  120  epsilon:  0.31
episode:  586  score:  288.24  avg_score:  39.43  frames:  395  epsilon:  0.31
episode:  587  score:  -48.16  avg_score:  38.21  frames:  192  epsilon:  0.31
episode:  588  score:  20.72  avg_score:  38.31  frames:  137  epsilon:  0.31
episode:  589  score:  122.59  avg_score:  40.53  frames:  999  epsilon:  0.31
episode:  590  score:  145.21  avg_score:  41.89  frames:  999  epsilon:  0.31
episode:  591  score:  125.70  avg_score:  44.66  frames:  999  epsilon:  0.31
episode:  592  score:  127.21  avg_score:  45.85  frames:  999  epsilon:  0.31
episode:  593  score:  79.17  avg_score:  46.51  frames:  999  epsilon:  0.31
episode:  594  score:  136.06  avg_score:  46.86  frames:  999  epsilon:  0.30
episode:  595  score:  18.69  avg_score:  47.26  frames:  86  epsilon:  0.30
episode:  596  score:  7.11  avg_score:  47.91  frames:  131  epsilon:  0.30
episode:  597  score:  58.99  avg_score:  48.65  frames:  999  epsilon:  0.30
episode:  598  score:  -60.12  avg_score:  48.13  frames:  999  epsilon:  0.30
episode:  599  score:  19.85  avg_score:  47.99  frames:  234  epsilon:  0.30
episode:  600  score:  63.78  avg_score:  49.50  frames:  999  epsilon:  0.30
episode:  601  score:  -0.38  avg_score:  48.41  frames:  119  epsilon:  0.30
episode:  602  score:  -0.07  avg_score:  48.62  frames:  94  epsilon:  0.30
episode:  603  score:  -49.86  avg_score:  48.15  frames:  197  epsilon:  0.30
episode:  604  score:  41.57  avg_score:  48.65  frames:  999  epsilon:  0.30
episode:  605  score:  -79.50  avg_score:  48.11  frames:  64  epsilon:  0.30
episode:  606  score:  39.12  avg_score:  48.87  frames:  112  epsilon:  0.30
episode:  607  score:  -12.06  avg_score:  48.36  frames:  238  epsilon:  0.30
episode:  608  score:  85.74  avg_score:  50.96  frames:  999  epsilon:  0.30
episode:  609  score:  115.60  avg_score:  52.99  frames:  999  epsilon:  0.30
episode:  610  score:  101.73  avg_score:  53.89  frames:  999  epsilon:  0.30
episode:  611  score:  78.78  avg_score:  55.51  frames:  999  epsilon:  0.29
episode:  612  score:  47.08  avg_score:  55.50  frames:  999  epsilon:  0.29
episode:  613  score:  52.93  avg_score:  53.36  frames:  999  epsilon:  0.29
episode:  614  score:  107.64  avg_score:  56.45  frames:  999  epsilon:  0.29
episode:  615  score:  -11.34  avg_score:  56.97  frames:  111  epsilon:  0.29
episode:  616  score:  131.75  avg_score:  60.83  frames:  999  epsilon:  0.29
episode:  617  score:  79.23  avg_score:  60.43  frames:  999  epsilon:  0.29
episode:  618  score:  58.29  avg_score:  60.34  frames:  999  epsilon:  0.29
episode:  619  score:  8.36  avg_score:  59.20  frames:  99  epsilon:  0.29
episode:  620  score:  -41.87  avg_score:  57.91  frames:  324  epsilon:  0.29
episode:  621  score:  -23.44  avg_score:  55.39  frames:  81  epsilon:  0.29
episode:  622  score:  233.36  avg_score:  58.04  frames:  849  epsilon:  0.29
episode:  623  score:  140.08  avg_score:  59.83  frames:  999  epsilon:  0.29
episode:  624  score:  75.35  avg_score:  59.84  frames:  999  epsilon:  0.29
episode:  625  score:  117.17  avg_score:  59.79  frames:  999  epsilon:  0.29
episode:  626  score:  132.37  avg_score:  61.00  frames:  999  epsilon:  0.29
episode:  627  score:  160.41  avg_score:  62.46  frames:  999  epsilon:  0.29
episode:  628  score:  115.39  avg_score:  63.84  frames:  999  epsilon:  0.28
episode:  629  score:  65.79  avg_score:  63.64  frames:  999  epsilon:  0.28
episode:  630  score:  109.76  avg_score:  63.83  frames:  999  epsilon:  0.28
episode:  631  score:  48.37  avg_score:  63.37  frames:  87  epsilon:  0.28
episode:  632  score:  -13.97  avg_score:  62.09  frames:  96  epsilon:  0.28
episode:  633  score:  241.58  avg_score:  63.70  frames:  356  epsilon:  0.28
episode:  634  score:  -9.02  avg_score:  62.93  frames:  106  epsilon:  0.28
episode:  635  score:  80.77  avg_score:  63.57  frames:  999  epsilon:  0.28
episode:  636  score:  35.88  avg_score:  62.25  frames:  151  epsilon:  0.28
episode:  637  score:  116.03  avg_score:  63.22  frames:  999  epsilon:  0.28
episode:  638  score:  -28.72  avg_score:  62.84  frames:  80  epsilon:  0.28
episode:  639  score:  23.66  avg_score:  62.22  frames:  98  epsilon:  0.28
episode:  640  score:  -0.86  avg_score:  62.20  frames:  999  epsilon:  0.28
episode:  641  score:  123.09  avg_score:  63.43  frames:  999  epsilon:  0.28
episode:  642  score:  -31.95  avg_score:  61.95  frames:  379  epsilon:  0.28
episode:  643  score:  -105.81  avg_score:  59.69  frames:  673  epsilon:  0.28
episode:  644  score:  121.35  avg_score:  58.45  frames:  999  epsilon:  0.28
episode:  645  score:  147.94  avg_score:  58.01  frames:  999  epsilon:  0.28
episode:  646  score:  193.66  avg_score:  60.13  frames:  814  epsilon:  0.27
episode:  647  score:  36.96  avg_score:  60.23  frames:  100  epsilon:  0.27
episode:  648  score:  106.86  avg_score:  60.06  frames:  999  epsilon:  0.27
episode:  649  score:  -7.50  avg_score:  58.25  frames:  92  epsilon:  0.27
episode:  650  score:  119.98  avg_score:  57.13  frames:  999  epsilon:  0.27
episode:  651  score:  109.95  avg_score:  58.95  frames:  999  epsilon:  0.27
episode:  652  score:  88.21  avg_score:  59.72  frames:  999  epsilon:  0.27
episode:  653  score:  -76.93  avg_score:  58.04  frames:  226  epsilon:  0.27
episode:  654  score:  221.10  avg_score:  59.63  frames:  508  epsilon:  0.27
episode:  655  score:  10.99  avg_score:  58.54  frames:  121  epsilon:  0.27
episode:  656  score:  -87.00  avg_score:  62.05  frames:  85  epsilon:  0.27
episode:  657  score:  172.34  avg_score:  62.91  frames:  315  epsilon:  0.27
episode:  658  score:  16.80  avg_score:  62.94  frames:  120  epsilon:  0.27
episode:  659  score:  -85.55  avg_score:  61.89  frames:  289  epsilon:  0.27
episode:  660  score:  19.43  avg_score:  61.95  frames:  102  epsilon:  0.27
episode:  661  score:  -5.04  avg_score:  60.36  frames:  152  epsilon:  0.27
episode:  662  score:  120.66  avg_score:  61.22  frames:  999  epsilon:  0.27
episode:  663  score:  -114.24  avg_score:  59.43  frames:  282  epsilon:  0.27
episode:  664  score:  -37.54  avg_score:  57.78  frames:  303  epsilon:  0.27
episode:  665  score:  97.21  avg_score:  57.96  frames:  999  epsilon:  0.26
episode:  666  score:  -11.54  avg_score:  58.56  frames:  177  epsilon:  0.26
episode:  667  score:  7.64  avg_score:  58.55  frames:  156  epsilon:  0.26
episode:  668  score:  50.03  avg_score:  56.53  frames:  999  epsilon:  0.26
episode:  669  score:  -76.12  avg_score:  55.28  frames:  174  epsilon:  0.26
episode:  670  score:  98.84  avg_score:  58.25  frames:  999  epsilon:  0.26
episode:  671  score:  45.88  avg_score:  57.23  frames:  999  epsilon:  0.26
episode:  672  score:  199.67  avg_score:  58.61  frames:  942  epsilon:  0.26
episode:  673  score:  120.78  avg_score:  58.23  frames:  999  epsilon:  0.26
episode:  674  score:  -123.28  avg_score:  56.69  frames:  186  epsilon:  0.26
episode:  675  score:  112.43  avg_score:  56.87  frames:  999  epsilon:  0.26
episode:  676  score:  -2.68  avg_score:  55.58  frames:  121  epsilon:  0.26
episode:  677  score:  103.76  avg_score:  59.78  frames:  999  epsilon:  0.26
episode:  678  score:  125.31  avg_score:  59.49  frames:  999  epsilon:  0.26
episode:  679  score:  113.11  avg_score:  59.84  frames:  999  epsilon:  0.26
episode:  680  score:  90.98  avg_score:  61.07  frames:  999  epsilon:  0.26
episode:  681  score:  103.11  avg_score:  60.96  frames:  999  epsilon:  0.26
episode:  682  score:  251.03  avg_score:  62.32  frames:  290  epsilon:  0.26
episode:  683  score:  144.59  avg_score:  61.04  frames:  999  epsilon:  0.26
episode:  684  score:  105.13  avg_score:  61.16  frames:  999  epsilon:  0.25
episode:  685  score:  121.87  avg_score:  62.13  frames:  999  epsilon:  0.25
episode:  686  score:  76.71  avg_score:  60.02  frames:  999  epsilon:  0.25
episode:  687  score:  228.62  avg_score:  62.79  frames:  511  epsilon:  0.25
episode:  688  score:  147.84  avg_score:  64.06  frames:  999  epsilon:  0.25
episode:  689  score:  115.86  avg_score:  63.99  frames:  999  epsilon:  0.25
episode:  690  score:  269.49  avg_score:  65.23  frames:  511  epsilon:  0.25
episode:  691  score:  138.97  avg_score:  65.36  frames:  999  epsilon:  0.25
episode:  692  score:  210.51  avg_score:  66.20  frames:  628  epsilon:  0.25
episode:  693  score:  18.27  avg_score:  65.59  frames:  148  epsilon:  0.25
episode:  694  score:  3.56  avg_score:  64.26  frames:  106  epsilon:  0.25
episode:  695  score:  120.05  avg_score:  65.28  frames:  999  epsilon:  0.25
episode:  696  score:  103.00  avg_score:  66.24  frames:  999  epsilon:  0.25
episode:  697  score:  114.98  avg_score:  66.80  frames:  999  epsilon:  0.25
episode:  698  score:  120.66  avg_score:  68.60  frames:  999  epsilon:  0.25
episode:  699  score:  141.82  avg_score:  69.82  frames:  999  epsilon:  0.25
episode:  700  score:  104.27  avg_score:  70.23  frames:  999  epsilon:  0.25
episode:  701  score:  38.50  avg_score:  70.62  frames:  126  epsilon:  0.25
episode:  702  score:  57.10  avg_score:  71.19  frames:  999  epsilon:  0.25
episode:  703  score:  64.14  avg_score:  72.33  frames:  999  epsilon:  0.25
episode:  704  score:  -181.55  avg_score:  70.10  frames:  720  epsilon:  0.24
episode:  705  score:  111.47  avg_score:  72.01  frames:  999  epsilon:  0.24
episode:  706  score:  71.01  avg_score:  72.33  frames:  999  epsilon:  0.24
episode:  707  score:  171.80  avg_score:  74.16  frames:  999  epsilon:  0.24
episode:  708  score:  -35.75  avg_score:  72.95  frames:  166  epsilon:  0.24
episode:  709  score:  255.04  avg_score:  74.34  frames:  728  epsilon:  0.24
episode:  710  score:  -212.92  avg_score:  71.20  frames:  218  epsilon:  0.24
episode:  711  score:  127.63  avg_score:  71.69  frames:  999  epsilon:  0.24
episode:  712  score:  91.71  avg_score:  72.13  frames:  999  epsilon:  0.24
episode:  713  score:  115.60  avg_score:  72.76  frames:  999  epsilon:  0.24
episode:  714  score:  196.40  avg_score:  73.65  frames:  534  epsilon:  0.24
episode:  715  score:  77.41  avg_score:  74.53  frames:  999  epsilon:  0.24
episode:  716  score:  96.17  avg_score:  74.18  frames:  999  epsilon:  0.24
episode:  717  score:  116.85  avg_score:  74.55  frames:  999  epsilon:  0.24
episode:  718  score:  169.08  avg_score:  75.66  frames:  668  epsilon:  0.24
episode:  719  score:  223.61  avg_score:  77.82  frames:  286  epsilon:  0.24
episode:  720  score:  243.62  avg_score:  80.67  frames:  972  epsilon:  0.24
episode:  721  score:  75.58  avg_score:  81.66  frames:  999  epsilon:  0.24
episode:  722  score:  194.10  avg_score:  81.27  frames:  608  epsilon:  0.24
episode:  723  score:  110.03  avg_score:  80.97  frames:  999  epsilon:  0.24
episode:  724  score:  50.45  avg_score:  80.72  frames:  999  epsilon:  0.24
episode:  725  score:  228.78  avg_score:  81.83  frames:  699  epsilon:  0.23
episode:  726  score:  68.68  avg_score:  81.20  frames:  999  epsilon:  0.23
episode:  727  score:  90.48  avg_score:  80.50  frames:  999  epsilon:  0.23
episode:  728  score:  -32.83  avg_score:  79.02  frames:  405  epsilon:  0.23
episode:  729  score:  144.01  avg_score:  79.80  frames:  859  epsilon:  0.23
episode:  730  score:  62.28  avg_score:  79.32  frames:  120  epsilon:  0.23
episode:  731  score:  249.68  avg_score:  81.34  frames:  282  epsilon:  0.23
episode:  732  score:  -104.10  avg_score:  80.44  frames:  479  epsilon:  0.23
episode:  733  score:  267.00  avg_score:  80.69  frames:  271  epsilon:  0.23
episode:  734  score:  210.24  avg_score:  82.88  frames:  419  epsilon:  0.23
episode:  735  score:  3.83  avg_score:  82.11  frames:  114  epsilon:  0.23
episode:  736  score:  -36.23  avg_score:  81.39  frames:  182  epsilon:  0.23
episode:  737  score:  125.18  avg_score:  81.48  frames:  999  epsilon:  0.23
episode:  738  score:  7.03  avg_score:  81.84  frames:  121  epsilon:  0.23
episode:  739  score:  151.95  avg_score:  83.12  frames:  999  epsilon:  0.23
episode:  740  score:  265.78  avg_score:  85.79  frames:  536  epsilon:  0.23
episode:  741  score:  97.29  avg_score:  85.53  frames:  999  epsilon:  0.23
episode:  742  score:  202.75  avg_score:  87.88  frames:  333  epsilon:  0.23
episode:  743  score:  211.26  avg_score:  91.05  frames:  386  epsilon:  0.23
episode:  744  score:  138.82  avg_score:  91.22  frames:  695  epsilon:  0.23
episode:  745  score:  -6.10  avg_score:  89.68  frames:  114  epsilon:  0.23
episode:  746  score:  205.64  avg_score:  89.80  frames:  355  epsilon:  0.22
episode:  747  score:  -5.19  avg_score:  89.38  frames:  88  epsilon:  0.22
episode:  748  score:  133.61  avg_score:  89.65  frames:  999  epsilon:  0.22
episode:  749  score:  115.43  avg_score:  90.88  frames:  999  epsilon:  0.22
episode:  750  score:  -44.05  avg_score:  89.24  frames:  86  epsilon:  0.22
episode:  751  score:  -113.09  avg_score:  87.01  frames:  64  epsilon:  0.22
episode:  752  score:  218.94  avg_score:  88.32  frames:  663  epsilon:  0.22
episode:  753  score:  21.59  avg_score:  89.30  frames:  157  epsilon:  0.22
episode:  754  score:  84.72  avg_score:  87.94  frames:  999  epsilon:  0.22
episode:  755  score:  -219.29  avg_score:  85.63  frames:  220  epsilon:  0.22
episode:  756  score:  -129.08  avg_score:  85.21  frames:  181  epsilon:  0.22
episode:  757  score:  -8.75  avg_score:  83.40  frames:  999  epsilon:  0.22
episode:  758  score:  269.24  avg_score:  85.93  frames:  776  epsilon:  0.22
episode:  759  score:  221.28  avg_score:  89.00  frames:  370  epsilon:  0.22
episode:  760  score:  -30.28  avg_score:  88.50  frames:  245  epsilon:  0.22
episode:  761  score:  116.23  avg_score:  89.71  frames:  999  epsilon:  0.22
episode:  762  score:  -54.59  avg_score:  87.96  frames:  999  epsilon:  0.22
episode:  763  score:  -139.06  avg_score:  87.71  frames:  233  epsilon:  0.22
episode:  764  score:  55.40  avg_score:  88.64  frames:  999  epsilon:  0.22
episode:  765  score:  -252.98  avg_score:  85.14  frames:  820  epsilon:  0.22
episode:  766  score:  -4.08  avg_score:  85.21  frames:  999  epsilon:  0.22
episode:  767  score:  51.84  avg_score:  85.65  frames:  999  epsilon:  0.22
episode:  768  score:  176.87  avg_score:  86.92  frames:  959  epsilon:  0.22
episode:  769  score:  -46.71  avg_score:  87.22  frames:  168  epsilon:  0.21
episode:  770  score:  184.54  avg_score:  88.07  frames:  544  epsilon:  0.21
episode:  771  score:  102.55  avg_score:  88.64  frames:  999  epsilon:  0.21
episode:  772  score:  199.50  avg_score:  88.64  frames:  493  epsilon:  0.21
episode:  773  score:  39.45  avg_score:  87.83  frames:  999  epsilon:  0.21
episode:  774  score:  19.55  avg_score:  89.25  frames:  999  epsilon:  0.21
episode:  775  score:  181.71  avg_score:  89.95  frames:  937  epsilon:  0.21
episode:  776  score:  121.58  avg_score:  91.19  frames:  999  epsilon:  0.21
episode:  777  score:  228.62  avg_score:  92.44  frames:  620  epsilon:  0.21
episode:  778  score:  227.32  avg_score:  93.46  frames:  356  epsilon:  0.21
episode:  779  score:  242.26  avg_score:  94.75  frames:  346  epsilon:  0.21
episode:  780  score:  243.96  avg_score:  96.28  frames:  522  epsilon:  0.21
episode:  781  score:  255.01  avg_score:  97.80  frames:  553  epsilon:  0.21
episode:  782  score:  -186.84  avg_score:  93.42  frames:  139  epsilon:  0.21
episode:  783  score:  85.77  avg_score:  92.83  frames:  999  epsilon:  0.21
episode:  784  score:  -38.91  avg_score:  91.39  frames:  175  epsilon:  0.21
episode:  785  score:  219.27  avg_score:  92.36  frames:  746  epsilon:  0.21
episode:  786  score:  224.35  avg_score:  93.84  frames:  549  epsilon:  0.21
episode:  787  score:  -94.55  avg_score:  90.61  frames:  151  epsilon:  0.21
episode:  788  score:  209.90  avg_score:  91.23  frames:  617  epsilon:  0.21
episode:  789  score:  223.04  avg_score:  92.30  frames:  811  epsilon:  0.21
episode:  790  score:  116.73  avg_score:  90.77  frames:  999  epsilon:  0.21
episode:  791  score:  248.85  avg_score:  91.87  frames:  812  epsilon:  0.21
episode:  792  score:  188.46  avg_score:  91.65  frames:  708  epsilon:  0.21
episode:  793  score:  113.25  avg_score:  92.60  frames:  999  epsilon:  0.20
episode:  794  score:  110.40  avg_score:  93.67  frames:  999  epsilon:  0.20
episode:  795  score:  117.46  avg_score:  93.64  frames:  999  epsilon:  0.20
episode:  796  score:  309.53  avg_score:  95.71  frames:  655  epsilon:  0.20
episode:  797  score:  108.77  avg_score:  95.65  frames:  999  epsilon:  0.20
episode:  798  score:  67.14  avg_score:  95.11  frames:  999  epsilon:  0.20
episode:  799  score:  108.99  avg_score:  94.78  frames:  999  epsilon:  0.20
episode:  800  score:  -55.99  avg_score:  93.18  frames:  101  epsilon:  0.20
episode:  801  score:  -34.20  avg_score:  92.45  frames:  114  epsilon:  0.20
episode:  802  score:  285.50  avg_score:  94.74  frames:  620  epsilon:  0.20
episode:  803  score:  190.11  avg_score:  96.00  frames:  713  epsilon:  0.20
episode:  804  score:  129.38  avg_score:  99.11  frames:  999  epsilon:  0.20
episode:  805  score:  278.11  avg_score:  100.77  frames:  407  epsilon:  0.20
episode:  806  score:  131.23  avg_score:  101.38  frames:  999  epsilon:  0.20
episode:  807  score:  165.33  avg_score:  101.31  frames:  774  epsilon:  0.20
episode:  808  score:  -177.80  avg_score:  99.89  frames:  635  epsilon:  0.20
episode:  809  score:  138.51  avg_score:  98.73  frames:  999  epsilon:  0.20
episode:  810  score:  -232.84  avg_score:  98.53  frames:  252  epsilon:  0.20
episode:  811  score:  215.13  avg_score:  99.40  frames:  388  epsilon:  0.20
episode:  812  score:  293.63  avg_score:  101.42  frames:  779  epsilon:  0.20
episode:  813  score:  119.15  avg_score:  101.46  frames:  999  epsilon:  0.20
episode:  814  score:  259.44  avg_score:  102.09  frames:  446  epsilon:  0.20
episode:  815  score:  160.34  avg_score:  102.92  frames:  612  epsilon:  0.20
episode:  816  score:  49.76  avg_score:  102.45  frames:  999  epsilon:  0.20
episode:  817  score:  -155.66  avg_score:  99.73  frames:  81  epsilon:  0.20
episode:  818  score:  75.36  avg_score:  98.79  frames:  999  epsilon:  0.19
episode:  819  score:  10.17  avg_score:  96.65  frames:  141  epsilon:  0.19
episode:  820  score:  113.74  avg_score:  95.36  frames:  792  epsilon:  0.19
episode:  821  score:  -24.43  avg_score:  94.36  frames:  96  epsilon:  0.19
episode:  822  score:  -41.12  avg_score:  92.00  frames:  999  epsilon:  0.19
episode:  823  score:  -141.65  avg_score:  89.49  frames:  87  epsilon:  0.19
episode:  824  score:  89.49  avg_score:  89.88  frames:  874  epsilon:  0.19
episode:  825  score:  182.00  avg_score:  89.41  frames:  627  epsilon:  0.19
episode:  826  score:  50.31  avg_score:  89.23  frames:  999  epsilon:  0.19
episode:  827  score:  -111.07  avg_score:  87.21  frames:  116  epsilon:  0.19
episode:  828  score:  241.98  avg_score:  89.96  frames:  648  epsilon:  0.19
episode:  829  score:  217.62  avg_score:  90.69  frames:  602  epsilon:  0.19
episode:  830  score:  132.36  avg_score:  91.40  frames:  999  epsilon:  0.19
episode:  831  score:  51.11  avg_score:  89.41  frames:  999  epsilon:  0.19
episode:  832  score:  210.49  avg_score:  92.56  frames:  490  epsilon:  0.19
episode:  833  score:  -218.76  avg_score:  87.70  frames:  226  epsilon:  0.19
episode:  834  score:  -16.30  avg_score:  85.43  frames:  999  epsilon:  0.19
episode:  835  score:  155.37  avg_score:  86.95  frames:  745  epsilon:  0.19
episode:  836  score:  200.39  avg_score:  89.31  frames:  475  epsilon:  0.19
episode:  837  score:  248.39  avg_score:  90.55  frames:  367  epsilon:  0.19
episode:  838  score:  215.50  avg_score:  92.63  frames:  760  epsilon:  0.19
episode:  839  score:  234.15  avg_score:  93.45  frames:  338  epsilon:  0.19
episode:  840  score:  229.14  avg_score:  93.09  frames:  821  epsilon:  0.19
episode:  841  score:  -52.66  avg_score:  91.59  frames:  155  epsilon:  0.19
episode:  842  score:  194.02  avg_score:  91.50  frames:  367  epsilon:  0.19
episode:  843  score:  100.61  avg_score:  90.39  frames:  999  epsilon:  0.19
episode:  844  score:  158.25  avg_score:  90.59  frames:  441  epsilon:  0.18
episode:  845  score:  264.24  avg_score:  93.29  frames:  339  epsilon:  0.18
episode:  846  score:  254.65  avg_score:  93.78  frames:  497  epsilon:  0.18
episode:  847  score:  210.46  avg_score:  95.94  frames:  400  epsilon:  0.18
episode:  848  score:  242.66  avg_score:  97.03  frames:  668  epsilon:  0.18
episode:  849  score:  174.09  avg_score:  97.61  frames:  738  epsilon:  0.18
episode:  850  score:  -218.68  avg_score:  95.87  frames:  212  epsilon:  0.18
episode:  851  score:  183.98  avg_score:  98.84  frames:  442  epsilon:  0.18
episode:  852  score:  73.91  avg_score:  97.39  frames:  999  epsilon:  0.18
episode:  853  score:  258.89  avg_score:  99.76  frames:  506  epsilon:  0.18
episode:  854  score:  244.41  avg_score:  101.36  frames:  431  epsilon:  0.18
episode:  855  score:  249.87  avg_score:  106.05  frames:  371  epsilon:  0.18
episode:  856  score:  235.49  avg_score:  109.70  frames:  566  epsilon:  0.18
episode:  857  score:  -4.18  avg_score:  109.74  frames:  110  epsilon:  0.18
episode:  858  score:  -182.24  avg_score:  105.23  frames:  85  epsilon:  0.18
episode:  859  score:  181.41  avg_score:  104.83  frames:  418  epsilon:  0.18
episode:  860  score:  81.87  avg_score:  105.95  frames:  999  epsilon:  0.18
episode:  861  score:  235.96  avg_score:  107.15  frames:  757  epsilon:  0.18
episode:  862  score:  215.56  avg_score:  109.85  frames:  518  epsilon:  0.18
episode:  863  score:  37.25  avg_score:  111.61  frames:  133  epsilon:  0.18
episode:  864  score:  17.97  avg_score:  111.24  frames:  121  epsilon:  0.18
episode:  865  score:  -1.22  avg_score:  113.76  frames:  112  epsilon:  0.18
episode:  866  score:  53.57  avg_score:  114.33  frames:  90  epsilon:  0.18
episode:  867  score:  170.85  avg_score:  115.52  frames:  601  epsilon:  0.18
episode:  868  score:  254.68  avg_score:  116.30  frames:  393  epsilon:  0.18
episode:  869  score:  221.53  avg_score:  118.98  frames:  351  epsilon:  0.18
episode:  870  score:  247.22  avg_score:  119.61  frames:  211  epsilon:  0.18
episode:  871  score:  232.79  avg_score:  120.91  frames:  396  epsilon:  0.18
episode:  872  score:  243.48  avg_score:  121.35  frames:  289  epsilon:  0.17
episode:  873  score:  227.99  avg_score:  123.24  frames:  726  epsilon:  0.17
episode:  874  score:  247.39  avg_score:  125.52  frames:  691  epsilon:  0.17
episode:  875  score:  228.91  avg_score:  125.99  frames:  243  epsilon:  0.17
episode:  876  score:  127.07  avg_score:  126.04  frames:  999  epsilon:  0.17
episode:  877  score:  195.82  avg_score:  125.71  frames:  732  epsilon:  0.17
episode:  878  score:  291.55  avg_score:  126.36  frames:  217  epsilon:  0.17
episode:  879  score:  228.33  avg_score:  126.22  frames:  788  epsilon:  0.17
episode:  880  score:  219.39  avg_score:  125.97  frames:  549  epsilon:  0.17
episode:  881  score:  5.01  avg_score:  123.47  frames:  121  epsilon:  0.17
episode:  882  score:  246.63  avg_score:  127.81  frames:  336  epsilon:  0.17
episode:  883  score:  245.17  avg_score:  129.40  frames:  408  epsilon:  0.17
episode:  884  score:  246.69  avg_score:  132.26  frames:  620  epsilon:  0.17
episode:  885  score:  -39.04  avg_score:  129.67  frames:  94  epsilon:  0.17
episode:  886  score:  261.70  avg_score:  130.05  frames:  212  epsilon:  0.17
episode:  887  score:  8.25  avg_score:  131.07  frames:  102  epsilon:  0.17
episode:  888  score:  98.77  avg_score:  129.96  frames:  999  epsilon:  0.17
episode:  889  score:  264.07  avg_score:  130.37  frames:  867  epsilon:  0.17
episode:  890  score:  47.78  avg_score:  129.68  frames:  80  epsilon:  0.17
episode:  891  score:  183.83  avg_score:  129.03  frames:  516  epsilon:  0.17
episode:  892  score:  -488.76  avg_score:  122.26  frames:  285  epsilon:  0.17
episode:  893  score:  26.60  avg_score:  121.39  frames:  138  epsilon:  0.17
episode:  894  score:  207.38  avg_score:  122.36  frames:  303  epsilon:  0.17
episode:  895  score:  -180.48  avg_score:  119.39  frames:  222  epsilon:  0.17
episode:  896  score:  218.68  avg_score:  118.48  frames:  379  epsilon:  0.17
episode:  897  score:  265.44  avg_score:  120.04  frames:  547  epsilon:  0.17
episode:  898  score:  239.15  avg_score:  121.76  frames:  743  epsilon:  0.17
episode:  899  score:  249.33  avg_score:  123.17  frames:  501  epsilon:  0.17
episode:  900  score:  -108.01  avg_score:  122.65  frames:  59  epsilon:  0.17
episode:  901  score:  244.24  avg_score:  125.43  frames:  379  epsilon:  0.17
episode:  902  score:  -111.66  avg_score:  121.46  frames:  85  epsilon:  0.16
episode:  903  score:  -11.55  avg_score:  119.44  frames:  140  epsilon:  0.16
episode:  904  score:  208.15  avg_score:  120.23  frames:  894  epsilon:  0.16
episode:  905  score:  221.61  avg_score:  119.67  frames:  563  epsilon:  0.16
episode:  906  score:  124.71  avg_score:  119.60  frames:  999  epsilon:  0.16
episode:  907  score:  181.59  avg_score:  119.76  frames:  439  epsilon:  0.16
episode:  908  score:  225.61  avg_score:  123.80  frames:  439  epsilon:  0.16
episode:  909  score:  218.44  avg_score:  124.60  frames:  563  epsilon:  0.16
episode:  910  score:  -125.50  avg_score:  125.67  frames:  548  epsilon:  0.16
episode:  911  score:  193.73  avg_score:  125.46  frames:  612  epsilon:  0.16
episode:  912  score:  -36.29  avg_score:  122.16  frames:  999  epsilon:  0.16
episode:  913  score:  305.08  avg_score:  124.02  frames:  359  epsilon:  0.16
episode:  914  score:  167.70  avg_score:  123.10  frames:  580  epsilon:  0.16
episode:  915  score:  283.28  avg_score:  124.33  frames:  315  epsilon:  0.16
episode:  916  score:  158.78  avg_score:  125.42  frames:  824  epsilon:  0.16
episode:  917  score:  205.12  avg_score:  129.03  frames:  461  epsilon:  0.16
episode:  918  score:  80.53  avg_score:  129.08  frames:  999  epsilon:  0.16
episode:  919  score:  164.32  avg_score:  130.62  frames:  621  epsilon:  0.16
episode:  920  score:  53.34  avg_score:  130.02  frames:  999  epsilon:  0.16
episode:  921  score:  134.87  avg_score:  131.61  frames:  755  epsilon:  0.16
episode:  922  score:  229.91  avg_score:  134.32  frames:  573  epsilon:  0.16
episode:  923  score:  227.79  avg_score:  138.01  frames:  833  epsilon:  0.16
episode:  924  score:  283.87  avg_score:  139.96  frames:  215  epsilon:  0.16
episode:  925  score:  268.51  avg_score:  140.82  frames:  212  epsilon:  0.16
episode:  926  score:  -31.58  avg_score:  140.00  frames:  93  epsilon:  0.16
episode:  927  score:  231.78  avg_score:  143.43  frames:  344  epsilon:  0.16
episode:  928  score:  209.32  avg_score:  143.10  frames:  727  epsilon:  0.16
episode:  929  score:  197.06  avg_score:  142.90  frames:  944  epsilon:  0.16
episode:  930  score:  153.66  avg_score:  143.11  frames:  999  epsilon:  0.16
episode:  931  score:  158.46  avg_score:  144.19  frames:  894  epsilon:  0.16
episode:  932  score:  216.19  avg_score:  144.24  frames:  918  epsilon:  0.16
episode:  933  score:  24.31  avg_score:  146.67  frames:  999  epsilon:  0.15
episode:  934  score:  250.77  avg_score:  149.34  frames:  797  epsilon:  0.15
episode:  935  score:  -70.05  avg_score:  147.09  frames:  91  epsilon:  0.15
episode:  936  score:  228.60  avg_score:  147.37  frames:  326  epsilon:  0.15
episode:  937  score:  233.94  avg_score:  147.23  frames:  502  epsilon:  0.15
episode:  938  score:  -22.19  avg_score:  144.85  frames:  240  epsilon:  0.15
episode:  939  score:  286.27  avg_score:  145.37  frames:  311  epsilon:  0.15
episode:  940  score:  -71.37  avg_score:  142.37  frames:  123  epsilon:  0.15
episode:  941  score:  238.93  avg_score:  145.28  frames:  260  epsilon:  0.15
episode:  942  score:  272.90  avg_score:  146.07  frames:  295  epsilon:  0.15
episode:  943  score:  224.25  avg_score:  147.31  frames:  215  epsilon:  0.15
episode:  944  score:  323.27  avg_score:  148.96  frames:  421  epsilon:  0.15
episode:  945  score:  270.39  avg_score:  149.02  frames:  386  epsilon:  0.15
episode:  946  score:  240.06  avg_score:  148.87  frames:  361  epsilon:  0.15
episode:  947  score:  -74.07  avg_score:  146.03  frames:  66  epsilon:  0.15
episode:  948  score:  45.27  avg_score:  144.05  frames:  93  epsilon:  0.15
episode:  949  score:  288.53  avg_score:  145.20  frames:  164  epsilon:  0.15
episode:  950  score:  21.95  avg_score:  147.60  frames:  78  epsilon:  0.15
episode:  951  score:  -962.53  avg_score:  136.14  frames:  161  epsilon:  0.15
episode:  952  score:  50.77  avg_score:  135.91  frames:  103  epsilon:  0.15
episode:  953  score:  4.77  avg_score:  133.37  frames:  86  epsilon:  0.15
episode:  954  score:  -34.37  avg_score:  130.58  frames:  90  epsilon:  0.15
episode:  955  score:  -134.04  avg_score:  126.74  frames:  75  epsilon:  0.15
episode:  956  score:  -77.06  avg_score:  123.61  frames:  76  epsilon:  0.15
episode:  957  score:  -102.96  avg_score:  122.63  frames:  63  epsilon:  0.15
episode:  958  score:  -124.05  avg_score:  123.21  frames:  78  epsilon:  0.15
episode:  959  score:  298.33  avg_score:  124.38  frames:  292  epsilon:  0.15
episode:  960  score:  258.82  avg_score:  126.15  frames:  249  epsilon:  0.15
episode:  961  score:  171.68  avg_score:  125.50  frames:  326  epsilon:  0.15
episode:  962  score:  252.61  avg_score:  125.88  frames:  291  epsilon:  0.15
episode:  963  score:  252.74  avg_score:  128.03  frames:  330  epsilon:  0.15
episode:  964  score:  -65.96  avg_score:  127.19  frames:  79  epsilon:  0.15
episode:  965  score:  231.38  avg_score:  129.52  frames:  594  epsilon:  0.15
episode:  966  score:  255.41  avg_score:  131.54  frames:  181  epsilon:  0.14
episode:  967  score:  -8.28  avg_score:  129.74  frames:  81  epsilon:  0.14
episode:  968  score:  153.58  avg_score:  128.73  frames:  647  epsilon:  0.14
episode:  969  score:  0.70  avg_score:  126.52  frames:  129  epsilon:  0.14
episode:  970  score:  182.67  avg_score:  125.88  frames:  255  epsilon:  0.14
episode:  971  score:  -207.33  avg_score:  121.48  frames:  83  epsilon:  0.14
episode:  972  score:  208.93  avg_score:  121.13  frames:  384  epsilon:  0.14
episode:  973  score:  260.18  avg_score:  121.45  frames:  345  epsilon:  0.14
episode:  974  score:  265.44  avg_score:  121.63  frames:  427  epsilon:  0.14
episode:  975  score:  273.00  avg_score:  122.08  frames:  545  epsilon:  0.14
episode:  976  score:  -44.12  avg_score:  120.36  frames:  145  epsilon:  0.14
episode:  977  score:  229.49  avg_score:  120.70  frames:  226  epsilon:  0.14
episode:  978  score:  282.47  avg_score:  120.61  frames:  325  epsilon:  0.14
episode:  979  score:  251.21  avg_score:  120.84  frames:  315  epsilon:  0.14
episode:  980  score:  43.13  avg_score:  119.08  frames:  167  epsilon:  0.14
episode:  981  score:  -14.87  avg_score:  118.88  frames:  154  epsilon:  0.14
episode:  982  score:  15.82  avg_score:  116.57  frames:  117  epsilon:  0.14
episode:  983  score:  142.92  avg_score:  115.55  frames:  889  epsilon:  0.14
episode:  984  score:  9.75  avg_score:  113.18  frames:  263  epsilon:  0.14
episode:  985  score:  -27.70  avg_score:  113.29  frames:  131  epsilon:  0.14
episode:  986  score:  295.94  avg_score:  113.63  frames:  235  epsilon:  0.14
episode:  987  score:  10.22  avg_score:  113.65  frames:  122  epsilon:  0.14
episode:  988  score:  260.43  avg_score:  115.27  frames:  421  epsilon:  0.14
episode:  989  score:  250.46  avg_score:  115.13  frames:  448  epsilon:  0.14
episode:  990  score:  123.97  avg_score:  115.90  frames:  998  epsilon:  0.14
episode:  991  score:  277.52  avg_score:  116.83  frames:  578  epsilon:  0.14
episode:  992  score:  -148.72  avg_score:  120.23  frames:  442  epsilon:  0.14
episode:  993  score:  245.37  avg_score:  122.42  frames:  524  epsilon:  0.14
episode:  994  score:  247.43  avg_score:  122.82  frames:  625  epsilon:  0.14
episode:  995  score:  269.87  avg_score:  127.32  frames:  421  epsilon:  0.14
episode:  996  score:  256.51  avg_score:  127.70  frames:  428  epsilon:  0.14
episode:  997  score:  -75.02  avg_score:  124.30  frames:  108  epsilon:  0.14
episode:  998  score:  235.33  avg_score:  124.26  frames:  227  epsilon:  0.14
episode:  999  score:  309.85  avg_score:  124.87  frames:  279  epsilon:  0.14
episode:  1000  score:  246.95  avg_score:  128.41  frames:  441  epsilon:  0.14
episode:  1001  score:  44.38  avg_score:  126.42  frames:  164  epsilon:  0.14
episode:  1002  score:  251.78  avg_score:  130.05  frames:  361  epsilon:  0.13
episode:  1003  score:  235.66  avg_score:  132.52  frames:  307  epsilon:  0.13
episode:  1004  score:  289.87  avg_score:  133.34  frames:  348  epsilon:  0.13
episode:  1005  score:  283.78  avg_score:  133.96  frames:  585  epsilon:  0.13
episode:  1006  score:  274.52  avg_score:  135.46  frames:  596  epsilon:  0.13
episode:  1007  score:  225.35  avg_score:  135.90  frames:  536  epsilon:  0.13
episode:  1008  score:  262.57  avg_score:  136.27  frames:  255  epsilon:  0.13
episode:  1009  score:  232.74  avg_score:  136.41  frames:  576  epsilon:  0.13
episode:  1010  score:  236.13  avg_score:  140.03  frames:  348  epsilon:  0.13
episode:  1011  score:  -243.59  avg_score:  135.65  frames:  187  epsilon:  0.13
episode:  1012  score:  276.48  avg_score:  138.78  frames:  431  epsilon:  0.13
episode:  1013  score:  267.52  avg_score:  138.41  frames:  282  epsilon:  0.13
episode:  1014  score:  219.02  avg_score:  138.92  frames:  622  epsilon:  0.13
episode:  1015  score:  220.37  avg_score:  138.29  frames:  537  epsilon:  0.13
episode:  1016  score:  248.25  avg_score:  139.18  frames:  318  epsilon:  0.13
episode:  1017  score:  -33.53  avg_score:  136.80  frames:  170  epsilon:  0.13
episode:  1018  score:  145.76  avg_score:  137.45  frames:  549  epsilon:  0.13
episode:  1019  score:  196.16  avg_score:  137.77  frames:  446  epsilon:  0.13
episode:  1020  score:  232.62  avg_score:  139.56  frames:  592  epsilon:  0.13
episode:  1021  score:  14.93  avg_score:  138.36  frames:  102  epsilon:  0.13
episode:  1022  score:  281.85  avg_score:  138.88  frames:  238  epsilon:  0.13
episode:  1023  score:  200.42  avg_score:  138.61  frames:  309  epsilon:  0.13
episode:  1024  score:  222.79  avg_score:  138.00  frames:  698  epsilon:  0.13
episode:  1025  score:  204.96  avg_score:  137.36  frames:  407  epsilon:  0.13
episode:  1026  score:  36.74  avg_score:  138.04  frames:  143  epsilon:  0.13
episode:  1027  score:  239.64  avg_score:  138.12  frames:  493  epsilon:  0.13
episode:  1028  score:  250.60  avg_score:  138.54  frames:  676  epsilon:  0.13
episode:  1029  score:  241.42  avg_score:  138.98  frames:  414  epsilon:  0.13
episode:  1030  score:  225.18  avg_score:  139.69  frames:  927  epsilon:  0.13
episode:  1031  score:  228.64  avg_score:  140.40  frames:  386  epsilon:  0.13
episode:  1032  score:  229.18  avg_score:  140.53  frames:  481  epsilon:  0.13
episode:  1033  score:  243.47  avg_score:  142.72  frames:  488  epsilon:  0.13
episode:  1034  score:  254.13  avg_score:  142.75  frames:  401  epsilon:  0.13
episode:  1035  score:  235.43  avg_score:  145.81  frames:  696  epsilon:  0.13
episode:  1036  score:  163.83  avg_score:  145.16  frames:  861  epsilon:  0.13
episode:  1037  score:  251.66  avg_score:  145.34  frames:  310  epsilon:  0.13
episode:  1038  score:  233.35  avg_score:  147.89  frames:  316  epsilon:  0.13
episode:  1039  score:  257.72  avg_score:  147.61  frames:  286  epsilon:  0.13
episode:  1040  score:  1.87  avg_score:  148.34  frames:  117  epsilon:  0.13
episode:  1041  score:  234.64  avg_score:  148.30  frames:  239  epsilon:  0.12
episode:  1042  score:  293.26  avg_score:  148.50  frames:  332  epsilon:  0.12
episode:  1043  score:  20.32  avg_score:  146.46  frames:  85  epsilon:  0.12
episode:  1044  score:  286.81  avg_score:  146.09  frames:  307  epsilon:  0.12
episode:  1045  score:  272.81  avg_score:  146.12  frames:  250  epsilon:  0.12
episode:  1046  score:  274.37  avg_score:  146.46  frames:  394  epsilon:  0.12
episode:  1047  score:  242.49  avg_score:  149.63  frames:  374  epsilon:  0.12
episode:  1048  score:  234.46  avg_score:  151.52  frames:  304  epsilon:  0.12
episode:  1049  score:  269.42  avg_score:  151.33  frames:  409  epsilon:  0.12
episode:  1050  score:  308.20  avg_score:  154.19  frames:  272  epsilon:  0.12
episode:  1051  score:  237.83  avg_score:  166.19  frames:  241  epsilon:  0.12
episode:  1052  score:  -6.52  avg_score:  165.62  frames:  123  epsilon:  0.12
episode:  1053  score:  268.88  avg_score:  168.26  frames:  313  epsilon:  0.12
episode:  1054  score:  267.99  avg_score:  171.29  frames:  281  epsilon:  0.12
episode:  1055  score:  26.52  avg_score:  172.89  frames:  122  epsilon:  0.12
episode:  1056  score:  284.87  avg_score:  176.51  frames:  234  epsilon:  0.12
episode:  1057  score:  263.29  avg_score:  180.17  frames:  269  epsilon:  0.12
episode:  1058  score:  11.09  avg_score:  181.53  frames:  117  epsilon:  0.12
episode:  1059  score:  298.73  avg_score:  181.53  frames:  275  epsilon:  0.12
episode:  1060  score:  292.32  avg_score:  181.86  frames:  811  epsilon:  0.12
episode:  1061  score:  -36.52  avg_score:  179.78  frames:  315  epsilon:  0.12
episode:  1062  score:  251.76  avg_score:  179.77  frames:  320  epsilon:  0.12
episode:  1063  score:  285.81  avg_score:  180.10  frames:  286  epsilon:  0.12
episode:  1064  score:  271.39  avg_score:  183.48  frames:  257  epsilon:  0.12
episode:  1065  score:  153.83  avg_score:  182.70  frames:  494  epsilon:  0.12
episode:  1066  score:  281.15  avg_score:  182.96  frames:  412  epsilon:  0.12
episode:  1067  score:  272.03  avg_score:  185.76  frames:  240  epsilon:  0.12
episode:  1068  score:  268.85  avg_score:  186.92  frames:  443  epsilon:  0.12
episode:  1069  score:  273.64  avg_score:  189.65  frames:  272  epsilon:  0.12
episode:  1070  score:  236.84  avg_score:  190.19  frames:  213  epsilon:  0.12
episode:  1071  score:  289.79  avg_score:  195.16  frames:  454  epsilon:  0.12
episode:  1072  score:  309.44  avg_score:  196.16  frames:  290  epsilon:  0.12
episode:  1073  score:  302.67  avg_score:  196.59  frames:  358  epsilon:  0.12
episode:  1074  score:  285.72  avg_score:  196.79  frames:  322  epsilon:  0.12
episode:  1075  score:  251.00  avg_score:  196.57  frames:  167  epsilon:  0.12
episode:  1076  score:  268.15  avg_score:  199.69  frames:  204  epsilon:  0.12
episode:  1077  score:  267.12  avg_score:  200.07  frames:  227  epsilon:  0.12
episode:  1078  score:  280.04  avg_score:  200.05  frames:  752  epsilon:  0.12
episode:  1079  score:  4.43  avg_score:  197.58  frames:  128  epsilon:  0.12
episode:  1080  score:  217.42  avg_score:  199.32  frames:  225  epsilon:  0.12
episode:  1081  score:  -22.31  avg_score:  199.25  frames:  123  epsilon:  0.12
episode:  1082  score:  264.58  avg_score:  201.73  frames:  210  epsilon:  0.11
episode:  1083  score:  14.80  avg_score:  200.45  frames:  166  epsilon:  0.11
episode:  1084  score:  203.71  avg_score:  202.39  frames:  481  epsilon:  0.11
episode:  1085  score:  264.14  avg_score:  205.31  frames:  337  epsilon:  0.11
episode:  1086  score:  210.81  avg_score:  204.46  frames:  459  epsilon:  0.11
episode:  1087  score:  192.55  avg_score:  206.28  frames:  312  epsilon:  0.11
episode:  1088  score:  288.56  avg_score:  206.56  frames:  235  epsilon:  0.11
episode:  1089  score:  238.59  avg_score:  206.44  frames:  256  epsilon:  0.11
episode:  1090  score:  53.36  avg_score:  205.74  frames:  128  epsilon:  0.11
episode:  1091  score:  225.30  avg_score:  205.22  frames:  189  epsilon:  0.11
episode:  1092  score:  225.59  avg_score:  208.96  frames:  308  epsilon:  0.11
episode:  1093  score:  288.82  avg_score:  209.39  frames:  194  epsilon:  0.11
episode:  1094  score:  263.05  avg_score:  209.55  frames:  247  epsilon:  0.11
episode:  1095  score:  123.14  avg_score:  208.08  frames:  999  epsilon:  0.11
episode:  1096  score:  256.21  avg_score:  208.08  frames:  245  epsilon:  0.11
episode:  1097  score:  277.83  avg_score:  211.61  frames:  526  epsilon:  0.11
episode:  1098  score:  -4.46  avg_score:  209.21  frames:  130  epsilon:  0.11
episode:  1099  score:  289.93  avg_score:  209.01  frames:  238  epsilon:  0.11
episode:  1100  score:  249.57  avg_score:  209.04  frames:  256  epsilon:  0.11
episode:  1101  score:  15.07  avg_score:  208.74  frames:  125  epsilon:  0.11
episode:  1102  score:  289.92  avg_score:  209.13  frames:  305  epsilon:  0.11
episode:  1103  score:  280.19  avg_score:  209.57  frames:  307  epsilon:  0.11
episode:  1104  score:  -26.71  avg_score:  206.41  frames:  134  epsilon:  0.11
episode:  1105  score:  7.50  avg_score:  203.64  frames:  216  epsilon:  0.11
episode:  1106  score:  284.51  avg_score:  203.74  frames:  231  epsilon:  0.11
episode:  1107  score:  217.57  avg_score:  203.66  frames:  258  epsilon:  0.11
episode:  1108  score:  238.35  avg_score:  203.42  frames:  297  epsilon:  0.11
episode:  1109  score:  279.87  avg_score:  203.89  frames:  233  epsilon:  0.11
episode:  1110  score:  265.32  avg_score:  204.19  frames:  268  epsilon:  0.11
episode:  1111  score:  280.89  avg_score:  209.43  frames:  358  epsilon:  0.11
episode:  1112  score:  267.24  avg_score:  209.34  frames:  239  epsilon:  0.11
episode:  1113  score:  204.09  avg_score:  208.70  frames:  326  epsilon:  0.11
episode:  1114  score:  231.96  avg_score:  208.83  frames:  198  epsilon:  0.11
episode:  1115  score:  139.87  avg_score:  208.03  frames:  499  epsilon:  0.11
episode:  1116  score:  219.51  avg_score:  207.74  frames:  270  epsilon:  0.11
episode:  1117  score:  269.94  avg_score:  210.78  frames:  360  epsilon:  0.11
episode:  1118  score:  228.94  avg_score:  211.61  frames:  459  epsilon:  0.11
episode:  1119  score:  268.04  avg_score:  212.33  frames:  470  epsilon:  0.11
episode:  1120  score:  292.05  avg_score:  212.92  frames:  397  epsilon:  0.11
episode:  1121  score:  268.42  avg_score:  215.46  frames:  309  epsilon:  0.11
episode:  1122  score:  222.02  avg_score:  214.86  frames:  252  epsilon:  0.11
episode:  1123  score:  303.65  avg_score:  215.89  frames:  365  epsilon:  0.11
episode:  1124  score:  303.57  avg_score:  216.70  frames:  272  epsilon:  0.11
episode:  1125  score:  253.41  avg_score:  217.18  frames:  365  epsilon:  0.11
episode:  1126  score:  291.24  avg_score:  219.73  frames:  325  epsilon:  0.11
episode:  1127  score:  257.63  avg_score:  219.91  frames:  360  epsilon:  0.11
episode:  1128  score:  217.18  avg_score:  219.57  frames:  366  epsilon:  0.10
episode:  1129  score:  293.07  avg_score:  220.09  frames:  335  epsilon:  0.10
episode:  1130  score:  285.42  avg_score:  220.69  frames:  262  epsilon:  0.10
episode:  1131  score:  208.13  avg_score:  220.49  frames:  367  epsilon:  0.10
episode:  1132  score:  237.30  avg_score:  220.57  frames:  258  epsilon:  0.10
episode:  1133  score:  274.61  avg_score:  220.88  frames:  218  epsilon:  0.10
episode:  1134  score:  269.40  avg_score:  221.03  frames:  236  epsilon:  0.10
episode:  1135  score:  273.27  avg_score:  221.41  frames:  271  epsilon:  0.10
episode:  1136  score:  281.67  avg_score:  222.59  frames:  233  epsilon:  0.10
episode:  1137  score:  255.06  avg_score:  222.62  frames:  275  epsilon:  0.10
episode:  1138  score:  227.68  avg_score:  222.57  frames:  272  epsilon:  0.10
episode:  1139  score:  304.10  avg_score:  223.03  frames:  217  epsilon:  0.10
episode:  1140  score:  273.73  avg_score:  225.75  frames:  305  epsilon:  0.10
episode:  1141  score:  258.25  avg_score:  225.98  frames:  211  epsilon:  0.10
episode:  1142  score:  309.31  avg_score:  226.14  frames:  224  epsilon:  0.10
episode:  1143  score:  284.28  avg_score:  228.78  frames:  349  epsilon:  0.10
episode:  1144  score:  269.30  avg_score:  228.61  frames:  313  epsilon:  0.10
episode:  1145  score:  287.65  avg_score:  228.76  frames:  226  epsilon:  0.10
episode:  1146  score:  280.63  avg_score:  228.82  frames:  239  epsilon:  0.10
episode:  1147  score:  280.34  avg_score:  229.20  frames:  222  epsilon:  0.10
episode:  1148  score:  247.78  avg_score:  229.33  frames:  511  epsilon:  0.10
episode:  1149  score:  290.44  avg_score:  229.54  frames:  213  epsilon:  0.10
episode:  1150  score:  230.00  avg_score:  228.76  frames:  544  epsilon:  0.10
episode:  1151  score:  265.74  avg_score:  229.04  frames:  212  epsilon:  0.10
episode:  1152  score:  288.39  avg_score:  231.99  frames:  229  epsilon:  0.10
episode:  1153  score:  259.21  avg_score:  231.89  frames:  290  epsilon:  0.10
episode:  1154  score:  -26.51  avg_score:  228.95  frames:  88  epsilon:  0.10
episode:  1155  score:  295.55  avg_score:  231.64  frames:  441  epsilon:  0.10
episode:  1156  score:  262.62  avg_score:  231.41  frames:  293  epsilon:  0.10
episode:  1157  score:  -77.82  avg_score:  228.00  frames:  84  epsilon:  0.10
episode:  1158  score:  260.17  avg_score:  230.49  frames:  381  epsilon:  0.10
episode:  1159  score:  278.71  avg_score:  230.29  frames:  431  epsilon:  0.10
episode:  1160  score:  262.72  avg_score:  230.00  frames:  367  epsilon:  0.10
episode:  1161  score:  12.70  avg_score:  230.49  frames:  139  epsilon:  0.10
episode:  1162  score:  287.24  avg_score:  230.84  frames:  283  epsilon:  0.10
episode:  1163  score:  259.59  avg_score:  230.58  frames:  241  epsilon:  0.10
episode:  1164  score:  40.19  avg_score:  228.27  frames:  98  epsilon:  0.10
episode:  1165  score:  237.58  avg_score:  229.11  frames:  312  epsilon:  0.10
episode:  1166  score:  264.85  avg_score:  228.94  frames:  316  epsilon:  0.10
episode:  1167  score:  257.74  avg_score:  228.80  frames:  240  epsilon:  0.10
episode:  1168  score:  298.70  avg_score:  229.10  frames:  339  epsilon:  0.10
episode:  1169  score:  254.18  avg_score:  228.91  frames:  438  epsilon:  0.10
episode:  1170  score:  275.07  avg_score:  229.29  frames:  193  epsilon:  0.10
episode:  1171  score:  -21.37  avg_score:  226.18  frames:  122  epsilon:  0.10
episode:  1172  score:  257.33  avg_score:  225.66  frames:  312  epsilon:  0.10
episode:  1173  score:  39.36  avg_score:  223.02  frames:  157  epsilon:  0.10
episode:  1174  score:  276.33  avg_score:  222.93  frames:  256  epsilon:  0.10
episode:  1175  score:  291.36  avg_score:  223.33  frames:  253  epsilon:  0.10
episode:  1176  score:  253.30  avg_score:  223.18  frames:  285  epsilon:  0.10
episode:  1177  score:  228.58  avg_score:  222.80  frames:  557  epsilon:  0.10
episode:  1178  score:  288.10  avg_score:  222.88  frames:  473  epsilon:  0.09
episode:  1179  score:  119.16  avg_score:  224.03  frames:  999  epsilon:  0.09
episode:  1180  score:  271.24  avg_score:  224.56  frames:  279  epsilon:  0.09
episode:  1181  score:  229.94  avg_score:  227.09  frames:  438  epsilon:  0.09
episode:  1182  score:  219.42  avg_score:  226.63  frames:  298  epsilon:  0.09
episode:  1183  score:  231.31  avg_score:  228.80  frames:  429  epsilon:  0.09
episode:  1184  score:  247.61  avg_score:  229.24  frames:  269  epsilon:  0.09
episode:  1185  score:  254.04  avg_score:  229.14  frames:  354  epsilon:  0.09
episode:  1186  score:  11.39  avg_score:  227.14  frames:  160  epsilon:  0.09
episode:  1187  score:  225.74  avg_score:  227.48  frames:  303  epsilon:  0.09
episode:  1188  score:  228.10  avg_score:  226.87  frames:  928  epsilon:  0.09
episode:  1189  score:  267.99  avg_score:  227.17  frames:  254  epsilon:  0.09
episode:  1190  score:  264.04  avg_score:  225.43  frames:  297  epsilon:  0.09
episode:  1191  score:  231.61  avg_score:  225.49  frames:  343  epsilon:  0.09
episode:  1192  score:  148.75  avg_score:  224.72  frames:  999  epsilon:  0.09
episode:  1193  score:  180.84  avg_score:  223.64  frames:  986  epsilon:  0.09
episode:  1194  score:  259.30  avg_score:  223.60  frames:  288  epsilon:  0.09
episode:  1195  score:  273.75  avg_score:  225.11  frames:  238  epsilon:  0.09
episode:  1196  score:  244.23  avg_score:  224.99  frames:  279  epsilon:  0.09
episode:  1197  score:  297.33  avg_score:  225.18  frames:  240  epsilon:  0.09
episode:  1198  score:  274.44  avg_score:  227.97  frames:  293  epsilon:  0.09
episode:  1199  score:  245.33  avg_score:  227.53  frames:  521  epsilon:  0.09
episode:  1200  score:  256.72  avg_score:  227.60  frames:  371  epsilon:  0.09
episode:  1201  score:  266.45  avg_score:  230.11  frames:  304  epsilon:  0.09
episode:  1202  score:  242.23  avg_score:  229.64  frames:  227  epsilon:  0.09
episode:  1203  score:  243.84  avg_score:  229.27  frames:  235  epsilon:  0.09
episode:  1204  score:  249.19  avg_score:  232.03  frames:  244  epsilon:  0.09
episode:  1205  score:  254.92  avg_score:  234.51  frames:  276  epsilon:  0.09
episode:  1206  score:  298.06  avg_score:  234.64  frames:  260  epsilon:  0.09
episode:  1207  score:  290.07  avg_score:  235.37  frames:  270  epsilon:  0.09
episode:  1208  score:  251.75  avg_score:  235.50  frames:  273  epsilon:  0.09
episode:  1209  score:  272.09  avg_score:  235.42  frames:  251  epsilon:  0.09
episode:  1210  score:  241.59  avg_score:  235.19  frames:  315  epsilon:  0.09
episode:  1211  score:  257.94  avg_score:  234.96  frames:  322  epsilon:  0.09
"""

In [27]:
max_e = 1
min_e=0.0001
decay_rate=0.002
learning_rate=0.0002
replay_batch_size=30
gamma = 0.99

with open('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(decay_rate)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':[float(i.split(" ")[6]) for i in string.split('\n') if i!='']}, fp)
